### OUR INITIAL DESGIGN TEMPLATE REFERENCE

In [20]:
"""
FUTURISTIC STUDENT ANALYTICS DASHBOARD
=====================================
Clean, aesthetic dashboard using only Plotly Dash & Bootstrap Components
No external CSS - pure component styling with perfect contrast
"""

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, State, callback
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta
import numpy as np

# ================================
# DATA LOADING & PROCESSING
# ================================
def load_and_process_data():
    """Generate mock data with correct column names - no real data loading"""
    print("📊 Generating mock data with correct column structure...")
    df_students, df_results, df_courses = generate_mock_data()
    
    # Calculate derived fields
    df_students = calculate_derived_fields(df_students, df_results)
    
    return df_students, df_results, df_courses

def generate_mock_data():
    """Generate realistic mock data with EXACT column names from your datasets"""
    np.random.seed(42)
    n_students = 800
    
    # Student Profiles - using YOUR actual column names
    students_data = {
        'STUDENT ID': [f'STU{i:05d}' for i in range(1, n_students + 1)],
        'GENDER': np.random.choice(['Male', 'Female'], n_students),
        'COUNTRY OF ORIGIN': np.random.choice(['Singapore', 'Malaysia', 'Indonesia', 'China', 'India'], n_students),
        'DOB': pd.date_range('1990-01-01', '2003-12-31', periods=n_students),
        'HIGHEST QUALIFICATION': np.random.choice(['Diploma', 'Bachelor', 'Master', 'PhD'], n_students, p=[0.3, 0.4, 0.25, 0.05]),
        'DATE ATTAINED HIGHEST QUALIFICATION': pd.date_range('2015-01-01', '2023-12-31', periods=n_students),
        'DESIGNATION': np.random.choice(['Student', 'Working Professional', 'Fresh Graduate'], n_students),
        'COMMENCEMENT DATE': pd.date_range('2020-01-01', '2024-01-01', periods=n_students),
        'COMPLETION DATE': pd.date_range('2022-01-01', '2026-12-31', periods=n_students),
        'FULL-TIME OR PART-TIME': np.random.choice(['Full-Time', 'Part-Time'], n_students, p=[0.7, 0.3]),
        'COURSE FUNDING': np.random.choice(['Self-Funded', 'Scholarship', 'Sponsored', 'Loan'], n_students),
        'CITIZENSHIP': np.random.choice(['Citizen', 'PR', 'International'], n_students, p=[0.6, 0.25, 0.15]),
        'COURSE CODE': np.random.choice(['CS101', 'BUS201', 'ENG301', 'MED401', 'ART501'], n_students),
        'DEPARTMENT': np.random.choice(['Computer Science', 'Business', 'Engineering', 'Medicine', 'Arts'], n_students)
    }
    
    df_students = pd.DataFrame(students_data)
    
    # Semester Results - using YOUR actual column names
    n_results = n_students * 4
    results_data = {
        'STUDENT ID': np.random.choice(df_students['STUDENT ID'], n_results),
        'PERIOD': np.random.choice(['2021 Sem 1', '2021 Sem 2', '2022 Sem 1', '2022 Sem 2', '2023 Sem 1', '2023 Sem 2'], n_results),
        'GPA': np.random.beta(2, 1, n_results) * 3 + 1,
        'COURSE CODE': np.random.choice(df_students['COURSE CODE'], n_results)
    }
    
    df_results = pd.DataFrame(results_data)
    df_results['GPA'] = df_results['GPA'].round(2)
    
    # Course Codes - using YOUR actual column names  
    courses_data = {
        'CODE': ['CS101', 'BUS201', 'ENG301', 'MED401', 'ART501'],
        'COURSE NAME': ['Computer Science', 'Business Studies', 'Engineering', 'Medicine', 'Arts & Design']
    }
    
    df_courses = pd.DataFrame(courses_data)
    
    return df_students, df_results, df_courses

def calculate_derived_fields(df_students, df_results):
    """Calculate CUMULATIVE_GPA, AGE_AT_QUALIFICATION using correct column names"""
    
    # Calculate CUMULATIVE_GPA
    cumulative_gpa = df_results.groupby('STUDENT ID')['GPA'].mean().reset_index()
    cumulative_gpa.columns = ['STUDENT ID', 'CUMULATIVE_GPA']
    df_students = df_students.merge(cumulative_gpa, on='STUDENT ID', how='left')
    df_students['CUMULATIVE_GPA'] = df_students['CUMULATIVE_GPA'].fillna(0).round(2)
    
    # Calculate AGE_AT_QUALIFICATION
    df_students['DOB'] = pd.to_datetime(df_students['DOB'])
    df_students['DATE ATTAINED HIGHEST QUALIFICATION'] = pd.to_datetime(df_students['DATE ATTAINED HIGHEST QUALIFICATION'])
    df_students['AGE_AT_QUALIFICATION'] = (
        df_students['DATE ATTAINED HIGHEST QUALIFICATION'] - df_students['DOB']
    ).dt.days // 365
    
    return df_students

# Load data
df_students, df_results, df_courses = load_and_process_data()

# ================================
# APP INITIALIZATION
# ================================
app = Dash(
    __name__,
    external_stylesheets=[dbc.themes.CYBORG],  # Futuristic dark theme
    suppress_callback_exceptions=True
)

app.title = "Student Analytics Dashboard"

# ================================
# FUTURISTIC GLOWING COLOR SCHEME
# ================================
COLORS = {
    'primary': '#8B5FBF',      # Purple gradient
    'secondary': '#22D3EE',    # Cyan blue  
    'accent': '#F97316',       # Orange accent
    'success': '#10B981',      # Green
    'warning': '#F59E0B',      # Amber
    'danger': '#EF4444',       # Red
    'pink': '#EC4899',         # Pink for cards
    'dark_bg': '#0A0A1A',      # Ultra dark background
    'card_bg': '#1A1A2E',      # Dark card background with glow
    'sidebar_bg': '#16213E',   # Glowing sidebar
    'light': '#FFFFFF',        # White
    'text': '#E2E8F0',         # Light gray text
    'text_dim': '#94A3B8',     # Dimmed text
    'neon_blue': '#00F5FF',    # Neon blue glow
    'neon_purple': '#BF40BF',  # Neon purple glow
    'neon_green': '#39FF14',   # Neon green glow
    'glow_shadow': 'rgba(34, 211, 238, 0.4)',  # Glowing shadow
    'cyan': '#22D3EE'          # Added cyan color for filter bar
}

# ================================
# DASHBOARD COMPONENTS
# ================================

# Header with enhanced glowing effects
header = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Div([
                    html.I(className="fas fa-graduation-cap", 
                          style={
                              'fontSize': '1.8rem', 
                              'color': COLORS['neon_blue'],
                              'filter': 'drop-shadow(0 0 8px rgba(0, 245, 255, 0.8))'
                          }),
                ], style={
                    'width': '50px', 'height': '50px', 'borderRadius': '15px',
                    'background': f'linear-gradient(135deg, {COLORS["primary"]} 0%, {COLORS["neon_blue"]} 100%)',
                    'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center',
                    'marginRight': '15px', 
                    'boxShadow': f'0 0 25px {COLORS["glow_shadow"]}, 0 0 50px rgba(139, 95, 191, 0.3)',
                    'border': f'1px solid {COLORS["neon_blue"]}',
                    'animation': 'pulse 2s infinite'
                }),
                html.Div([
                    html.H5("STUDENT ANALYTICS", style={
                        'color': COLORS['light'], 'margin': '0', 'fontWeight': '700',
                        'letterSpacing': '1px', 'fontSize': '0.95rem',
                        'textShadow': f'0 0 10px {COLORS["neon_blue"]}',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    html.P("CONTROL CENTER", style={
                        'color': COLORS['neon_blue'], 'margin': '0', 'fontSize': '0.75rem',
                        'fontWeight': '500', 'letterSpacing': '0.5px',
                        'fontFamily': 'Rajdhani, monospace'
                    })
                ])
            ], style={'display': 'flex', 'alignItems': 'center'})
        ], width=3),
        
        dbc.Col([
            html.H1("ANALYTICS NEXUS", 
                   className="text-center",
                   style={
                       'color': COLORS['light'], 
                       'fontWeight': '900',
                       'fontSize': '2.5rem',
                       'margin': '0',
                       'background': f'linear-gradient(135deg, {COLORS["neon_blue"]} 0%, {COLORS["neon_purple"]} 50%, {COLORS["primary"]} 100%)',
                       'backgroundClip': 'text',
                       'WebkitBackgroundClip': 'text',
                       'WebkitTextFillColor': 'transparent',
                       'letterSpacing': '2px',
                       'textShadow': '0 0 20px rgba(34, 211, 238, 0.6)',
                       'filter': 'drop-shadow(0 0 10px rgba(34, 211, 238, 0.8))',
                       'fontFamily': 'Orbitron, monospace'
                   })
        ], width=6),
        
        dbc.Col([
            html.Div([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-circle", style={
                            'color': COLORS['neon_green'], 'fontSize': '0.6rem',
                            'animation': 'blink 1.5s infinite'
                        }),
                        html.Span("LIVE", style={
                            'color': COLORS['neon_green'], 'fontSize': '0.8rem', 
                            'marginLeft': '6px', 'fontWeight': '700',
                            'textShadow': f'0 0 8px {COLORS["neon_green"]}',
                            'fontFamily': 'Orbitron, monospace'
                        })
                    ], style={'display': 'flex', 'alignItems': 'center', 'marginBottom': '4px'}),
                    html.Span(id="live-time", style={
                        'color': COLORS['text'], 'fontSize': '0.85rem',
                        'fontFamily': 'Rajdhani, monospace', 'fontWeight': '500'
                    })
                ], style={'textAlign': 'right'})
            ])
        ], width=3)
    ], className="align-items-center"),
], fluid=True, className="py-4", 
   style={
       'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)', 
       'marginBottom': '30px',
       'borderRadius': '0 0 30px 30px',
       'boxShadow': f'0 0 40px {COLORS["glow_shadow"]}, 0 15px 60px rgba(26, 26, 46, 0.8)',
       'border': f'1px solid {COLORS["neon_blue"]}20',
       'position': 'relative',
       'overflow': 'hidden'
   })

# Navigation/Filter Bar with enhanced glowing effects - simplified to search only
navbar = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Label("SEARCH STUDENT ID", style={
                    'color': COLORS['pink'], 'fontWeight': '700', 'marginBottom': '10px',
                    'fontSize': '0.85rem', 'letterSpacing': '1px',
                    'textShadow': f'0 0 8px {COLORS["pink"]}',
                    'fontFamily': 'Orbitron, monospace'
                }),
                dbc.Input(
                    id='search-box',
                    placeholder="Enter Student ID (e.g., STU00001)",
                    type="text",
                    style={
                        'backgroundColor': COLORS['sidebar_bg'], 'color': COLORS['light'], 
                        'border': f'2px solid {COLORS["pink"]}60', 'borderRadius': '10px',
                        'boxShadow': f'inset 0 0 10px rgba(236, 72, 153, 0.2)',
                        'fontFamily': 'Rajdhani, monospace'
                    }
                )
            ], style={
                'backgroundColor': COLORS['card_bg'], 'padding': '20px', 'borderRadius': '20px',
                'boxShadow': f'0 0 30px rgba(236, 72, 153, 0.4), 0 8px 32px rgba(26, 26, 46, 0.6)', 
                'border': f'2px solid {COLORS["pink"]}40',
                'backdropFilter': 'blur(10px)',
                'transition': 'all 0.3s ease'
            })
        ], width=6, className="mx-auto")  # Centered column
    ], className="justify-content-center")
], fluid=True, className="py-3", style={'marginBottom': '30px', 'position': 'relative', 'zIndex': '5'})

# KPI Cards with enhanced glowing effects
def create_kpi_card(title, value, subtitle, icon, color, gradient_color):
    return dbc.Col([
        html.Div([
            html.Div([
                # Glowing icon with enhanced effects
                html.Div([
                    html.I(className=icon, style={
                        'fontSize': '2rem', 'color': 'white',
                        'filter': f'drop-shadow(0 0 10px {color})'
                    })
                ], style={
                    'width': '70px', 'height': '70px', 'borderRadius': '20px',
                    'background': f'linear-gradient(135deg, {color} 0%, {gradient_color} 100%)',
                    'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center',
                    'boxShadow': f'0 0 30px {color}60, 0 0 60px {color}30',
                    'border': f'2px solid {color}80',
                    'animation': 'glow 3s ease-in-out infinite alternate'
                }),
                
                # Enhanced text content
                html.Div([
                    html.H2(value, style={
                        'color': COLORS['light'], 'fontWeight': '900', 'fontSize': '2.4rem', 
                        'margin': '10px 0 6px 0', 'lineHeight': '1',
                        'textShadow': f'0 0 15px {color}80',
                        'letterSpacing': '1px',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    html.H6(title, style={
                        'color': color, 'fontWeight': '700', 'margin': '0 0 6px 0',
                        'fontSize': '1.1rem', 'letterSpacing': '0.5px',
                        'textShadow': f'0 0 8px {color}60',
                        'fontFamily': 'Orbitron, monospace',
                        'textTransform': 'uppercase'
                    }),
                    html.P(subtitle, style={
                        'color': COLORS['text_dim'], 'fontSize': '0.9rem', 'margin': '0',
                        'fontWeight': '500', 'fontFamily': 'Rajdhani, monospace'
                    })
                ], style={'flex': '1', 'marginLeft': '20px'})
            ], style={'display': 'flex', 'alignItems': 'center'})
        ], style={
            'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)',
            'border': f'2px solid {color}30',
            'borderRadius': '25px',
            'padding': '30px',
            'boxShadow': f'0 0 40px {color}20, 0 15px 60px rgba(26, 26, 46, 0.8)',
            'transition': 'all 0.3s ease',
            'height': '100%',
            'backdropFilter': 'blur(15px)',
            'position': 'relative',
            'overflow': 'hidden'
        })
    ], width=3, className="mb-4")

kpi_section = html.Div(id="kpi-cards-left")  # Updated for left sidebar

# Chart containers with enhanced glowing design
chart_style = {
    'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)',
    'border': f'2px solid {COLORS["neon_blue"]}40',
    'borderRadius': '25px',
    'padding': '20px',  # Reduced from 30px
    'marginBottom': '20px',  # Reduced from 30px
    'boxShadow': f'0 0 40px {COLORS["glow_shadow"]}, 0 15px 60px rgba(26, 26, 46, 0.8)',
    'backdropFilter': 'blur(15px)',
    'position': 'relative',
    'zIndex': '1',
    'overflow': 'hidden',
    'transition': 'all 0.3s ease'
}

charts_section = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("PERFORMANCE METRICS", 
                       style={
                           'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                           'textTransform': 'uppercase'
                       }),
                dcc.Graph(id='gpa-by-dept-chart', config={'displayModeBar': False})
            ], style=chart_style)
        ], width=6),
        
        dbc.Col([
            html.Div([
                html.H4("DISTRIBUTION ANALYSIS", 
                       style={
                           'color': COLORS['neon_purple'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["neon_purple"]}, 0 0 30px {COLORS["neon_purple"]}40',
                           'textTransform': 'uppercase'
                       }),
                dcc.Graph(id='citizenship-chart', config={'displayModeBar': False})
            ], style=chart_style)
        ], width=6)
    ], className="g-4"),
    
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("TEMPORAL TRENDS", 
                       style={
                           'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                           'textTransform': 'uppercase'
                       }),
                dcc.Graph(id='gpa-trend-chart', config={'displayModeBar': False})
            ], style=chart_style)
        ], width=6),
        
        dbc.Col([
            html.Div([
                html.H4("CORRELATION MATRIX", 
                       style={
                           'color': COLORS['accent'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["accent"]}, 0 0 30px {COLORS["accent"]}40',
                           'textTransform': 'uppercase'
                       }),
                dcc.Graph(id='gpa-age-scatter', config={'displayModeBar': False})
            ], style=chart_style)
        ], width=6)
    ], className="g-4")
], fluid=True)

# ================================
# MAIN LAYOUT WITH LEFT-RIGHT ORIENTATION
# ================================
app.layout = html.Div([
    # Global CSS for futuristic fonts
    html.Link(
        rel='stylesheet',
        href='https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&family=Rajdhani:wght@300;400;500;600;700&family=Space+Mono:wght@400;700&display=swap'
    ),
    
    # Live update interval
    dcc.Interval(id='interval-update', interval=60000, n_intervals=0),
    
    # Data store
    dcc.Store(id='filtered-data'),
    
    # Header (full width)
    header,
    
    # Main Content Container with Left-Right Layout
    dbc.Container([
        dbc.Row([
            # LEFT SIDE - Search, Filters, Student Data, and KPIs (4/12 columns)
            dbc.Col([
                # Search Section
                html.Div([
                    html.Label("🔍 SEARCH STUDENT ID", style={
                        'color': COLORS['pink'], 'fontWeight': '700', 'marginBottom': '15px',
                        'fontSize': '1rem', 'letterSpacing': '1px',
                        'textShadow': f'0 0 8px {COLORS["pink"]}',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    dbc.Input(
                        id='search-box',
                        placeholder="Enter Student ID (e.g., STU00001)",
                        type="text",
                        style={
                            'backgroundColor': COLORS['sidebar_bg'], 'color': COLORS['light'], 
                            'border': f'2px solid {COLORS["pink"]}60', 'borderRadius': '10px',
                            'boxShadow': f'inset 0 0 10px rgba(236, 72, 153, 0.2)',
                            'fontFamily': 'Rajdhani, monospace',
                            'fontSize': '1rem'
                        }
                    )
                ], style={
                    'backgroundColor': COLORS['card_bg'], 'padding': '20px', 'borderRadius': '20px',
                    'boxShadow': f'0 0 30px rgba(236, 72, 153, 0.4), 0 8px 32px rgba(26, 26, 46, 0.6)', 
                    'border': f'2px solid {COLORS["pink"]}40',
                    'backdropFilter': 'blur(10px)',
                    'marginBottom': '20px'
                }),
                
                # Student Data Section
                html.Div([
                    html.H4("📊 STUDENT DATA", 
                           style={
                               'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                               'textTransform': 'uppercase'
                           }),
                    html.Div(id='student-data-table')
                ], style={**chart_style, 'height': '300px', 'overflow': 'auto', 'marginBottom': '20px'}),
                
                # KPI Cards Section (Compact vertical layout)
                html.Div([
                    html.H4("📈 KEY METRICS", 
                           style={
                               'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                               'textTransform': 'uppercase'
                           }),
                    html.Div(id='kpi-cards-left')
                ], style=chart_style)
                
            ], width=4),  # Left side takes 4/12 columns
            
            # RIGHT SIDE - Scrollable Charts (8/12 columns)
            dbc.Col([
                # Simple Date Filter Row
                html.Div([
                    html.Span("📅 Filter: ", style={
                        'color': COLORS['cyan'], 'fontWeight': '700', 'marginRight': '10px',
                        'fontFamily': 'Orbitron, monospace', 'fontSize': '0.9rem'
                    }),
                    dcc.DatePickerSingle(
                        id='start-date-picker',
                        date=datetime(2020, 1, 1),
                        display_format='YYYY-MM-DD',
                        style={'display': 'inline-block', 'marginRight': '10px'}
                    ),
                    html.Span(" to ", style={'color': COLORS['light'], 'margin': '0 5px'}),
                    dcc.DatePickerSingle(
                        id='end-date-picker',
                        date=datetime(2026, 12, 31),
                        display_format='YYYY-MM-DD',
                        style={'display': 'inline-block', 'marginRight': '15px'}
                    ),
                    dbc.Button("All Time", id="btn-all-years", color="outline-info", size="sm",
                              style={'fontFamily': 'Rajdhani, monospace', 'fontSize': '0.8rem'})
                ], style={
                    'backgroundColor': COLORS['card_bg'], 'padding': '10px 15px', 'borderRadius': '10px',
                    'border': f'1px solid {COLORS["cyan"]}40', 'marginBottom': '15px',
                    'display': 'flex', 'alignItems': 'center', 'flexWrap': 'wrap'
                }),
                
                # Scrollable Charts Container
                html.Div([
                    # Chart 1 - Performance Metrics
                    html.Div([
                        html.H4("PERFORMANCE METRICS", 
                               style={
                                   'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='gpa-by-dept-chart', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 2 - Distribution Analysis
                    html.Div([
                        html.H4("DISTRIBUTION ANALYSIS", 
                               style={
                                   'color': COLORS['neon_purple'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_purple"]}, 0 0 30px {COLORS["neon_purple"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='citizenship-chart', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 3 - Temporal Trends
                    html.Div([
                        html.H4("TEMPORAL TRENDS", 
                               style={
                                   'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='gpa-trend-chart', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 4 - Correlation Analysis
                    html.Div([
                        html.H4("CORRELATION ANALYSIS", 
                               style={
                                   'color': COLORS['accent'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["accent"]}, 0 0 30px {COLORS["accent"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='gpa-age-scatter', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style=chart_style)
                    
                ], style={
                    'height': '90vh',  # Set viewport height
                    'overflowY': 'auto',  # Enable vertical scrolling
                    'paddingRight': '15px',  # Space for scrollbar
                    'scrollbarWidth': 'thin',  # Thin scrollbar for Firefox
                    'scrollbarColor': f'{COLORS["neon_blue"]} {COLORS["sidebar_bg"]}'  # Custom scrollbar colors
                })
                
            ], width=8)  # Right side takes 8/12 columns
            
        ], className="g-4")
    ], fluid=True)
], style={
    'backgroundColor': COLORS['dark_bg'], 
    'minHeight': '100vh', 
    'padding': '0',
    'fontFamily': 'Rajdhani, Orbitron, monospace'
})

# Add custom CSS for date picker through app.index_string
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            /* Custom scrollbar styling for chart container */
            ::-webkit-scrollbar {
                width: 12px;
            }
            ::-webkit-scrollbar-track {
                background: #1A1A2E;
                border-radius: 10px;
            }
            ::-webkit-scrollbar-thumb {
                background: linear-gradient(135deg, #00F5FF, #BF40BF);
                border-radius: 10px;
                border: 2px solid #1A1A2E;
            }
            ::-webkit-scrollbar-thumb:hover {
                background: linear-gradient(135deg, #22D3EE, #8B5FBF);
                box-shadow: 0 0 10px rgba(0, 245, 255, 0.6);
            }
            
            /* Custom Dropdown Styling */
            .custom-dropdown .Select-control {
                background-color: #16213E !important;
                border: 2px solid #00F5FF60 !important;
                border-radius: 8px !important;
                color: #FFFFFF !important;
            }
            .custom-dropdown .Select-menu-outer {
                background-color: #1A1A2E !important;
                border: 2px solid #00F5FF !important;
                border-radius: 10px !important;
                box-shadow: 0 0 20px rgba(0, 245, 255, 0.4) !important;
            }
            .custom-dropdown .Select-option {
                background-color: #16213E !important;
                color: #FFFFFF !important;
                padding: 10px !important;
            }
            .custom-dropdown .Select-option:hover {
                background-color: #22D3EE !important;
                color: #000000 !important;
            }
            .custom-dropdown .Select-value-label {
                color: #FFFFFF !important;
            }
            .custom-dropdown .Select-arrow-zone {
                padding-right: 10px !important;
            }
            .custom-dropdown .Select-arrow {
                border-color: #00F5FF transparent transparent !important;
            }
            
            /* Radio Items and Checklist Styling */
            .dash-checkbox input[type="checkbox"] {
                accent-color: #00F5FF !important;
                transform: scale(1.2);
            }
            .dash-radioitems input[type="radio"] {
                accent-color: #39FF14 !important;
                transform: scale(1.2);
            }
            
            /* Date picker modal-style positioning */
            .DateInput_input {
                z-index: 10 !important;
                position: relative !important;
                background: #16213E !important;
                color: #FFFFFF !important;
                border: 2px solid #00F5FF !important;
                border-radius: 8px !important;
            }
            .DateRangePickerInput {
                background: transparent !important;
                border: none !important;
            }
            .DateRangePicker {
                position: relative !important;
                z-index: 10 !important;
            }
            .DateInput {
                position: relative !important;
                z-index: 10 !important;
            }
            .SingleDatePicker {
                position: relative !important;
                z-index: 10 !important;
            }
            .DateRangePicker__picker {
                position: absolute !important;
                top: 100% !important;
                left: 0 !important;
                z-index: 9999 !important;
                background: #1A1A2E !important;
                border: 2px solid #00F5FF !important;
                border-radius: 15px !important;
                box-shadow: 0 0 40px rgba(0, 245, 255, 0.6) !important;
                margin-top: 8px !important;
                transform: none !important;
            }
            .DateRangePicker__picker--direction-up {
                top: auto !important;
                bottom: 100% !important;
                margin-bottom: 8px !important;
                margin-top: 0 !important;
            }
            .DateRangePicker__picker--direction-down {
                top: 100% !important;
                bottom: auto !important;
                margin-top: 8px !important;
                margin-bottom: 0 !important;
            }
            .DayPicker {
                background: #1A1A2E !important;
            }
            .DayPicker__wrapper {
                background: transparent !important;
            }
            .CalendarDay {
                background: #16213E !important;
                color: #FFFFFF !important;
                border: 1px solid #00F5FF20 !important;
            }
            .CalendarDay:hover {
                background: #22D3EE !important;
                color: #000000 !important;
            }
            .CalendarDay__selected {
                background: #00F5FF !important;
                color: #000000 !important;
            }
            .CalendarDay__selected:hover {
                background: #00F5FF !important;
                color: #000000 !important;
            }
            .CalendarDay__hovered_span {
                background: #22D3EE !important;
                color: #000000 !important;
            }
            .CalendarDay__blocked_out_of_range {
                background: #0A0A1A !important;
                color: #666 !important;
            }
            .DayPickerNavigation_button {
                background: #16213E !important;
                border: 2px solid #00F5FF !important;
                color: #00F5FF !important;
            }
            .DayPickerNavigation_button:hover {
                background: #00F5FF !important;
                color: #000000 !important;
            }
            .DayPicker_weekHeader {
                color: #00F5FF !important;
            }
            .CalendarMonth_caption {
                color: #00F5FF !important;
                font-weight: bold !important;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# ================================
# CALLBACKS
# ================================

# Update live time
@app.callback(
    Output('live-time', 'children'),
    Input('interval-update', 'n_intervals')
)
def update_time(n):
    return datetime.now().strftime("Last Updated: %Y-%m-%d %H:%M:%S")

# Date filter quick buttons callbacks
@app.callback(
    [Output('start-date-picker', 'date'),
     Output('end-date-picker', 'date')],
    [Input('btn-all-years', 'n_clicks')],
    prevent_initial_call=True
)
def update_date_range(btn_all):
    from dash import callback_context
    ctx = callback_context
    if not ctx.triggered:
        return datetime(2020, 1, 1), datetime(2026, 12, 31)
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == 'btn-all-years':
        return datetime(2020, 1, 1), datetime(2026, 12, 31)
    
    return datetime(2020, 1, 1), datetime(2026, 12, 31)

# Filter data based on search box AND date range - UPDATED
@app.callback(
    Output('filtered-data', 'data'),
    [Input('search-box', 'value'),
     Input('start-date-picker', 'date'),
     Input('end-date-picker', 'date')]
)
def filter_data(search, start_date, end_date):
    filtered_df = df_students.copy()
    
    # Search filter
    if search:
        filtered_df = filtered_df[
            filtered_df['STUDENT ID'].str.contains(search, case=False, na=False)
        ]
    
    # Date range filter - filter by COMMENCEMENT DATE
    if start_date and end_date:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        
        # Convert COMMENCEMENT DATE to datetime if it's not already
        filtered_df['COMMENCEMENT DATE'] = pd.to_datetime(filtered_df['COMMENCEMENT DATE'], errors='coerce')
        
        # Filter by date range
        filtered_df = filtered_df[
            (filtered_df['COMMENCEMENT DATE'] >= start_date) & 
            (filtered_df['COMMENCEMENT DATE'] <= end_date)
        ]
    
    return filtered_df.to_dict('records')

# Update KPI cards - show filtered data statistics
@app.callback(
    Output('kpi-cards-left', 'children'),
    Input('filtered-data', 'data')
)
def update_kpi_cards(filtered_data):
    if not filtered_data:
        # Show empty state
        return html.Div([
            html.P("No data matches current filters", 
                   style={
                       'textAlign': 'center', 
                       'color': COLORS['text_dim'], 
                       'fontSize': '1.1rem',
                       'fontFamily': 'Rajdhani, monospace',
                       'marginTop': '40px'
                   })
        ])
    
    df = pd.DataFrame(filtered_data)
    
    total_students = len(df)
    avg_gpa = df['CUMULATIVE_GPA'].mean()
    avg_age = df['AGE_AT_QUALIFICATION'].mean()
    citizenship_counts = df['CITIZENSHIP'].value_counts()
    
    return html.Div([
        # Compact KPI cards for left sidebar
        html.Div([
            html.Div([
                html.I(className="fas fa-users", style={
                    'fontSize': '1.5rem', 'color': COLORS['primary'],
                    'marginRight': '12px'
                }),
                html.Div([
                    html.H5(f"{total_students:,}", style={
                        'color': COLORS['light'], 'margin': '0', 'fontWeight': '700',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    html.P("Filtered Students", style={
                        'color': COLORS['primary'], 'margin': '0', 'fontSize': '0.85rem',
                        'fontFamily': 'Rajdhani, monospace'
                    })
                ])
            ], style={
                'display': 'flex', 'alignItems': 'center', 'padding': '15px',
                'backgroundColor': COLORS['sidebar_bg'], 'borderRadius': '10px',
                'marginBottom': '10px', 'border': f'1px solid {COLORS["primary"]}40'
            }),
            
            html.Div([
                html.I(className="fas fa-chart-line", style={
                    'fontSize': '1.5rem', 'color': COLORS['success'],
                    'marginRight': '12px'
                }),
                html.Div([
                    html.H5(f"{avg_gpa:.2f}", style={
                        'color': COLORS['light'], 'margin': '0', 'fontWeight': '700',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    html.P("Average GPA", style={
                        'color': COLORS['success'], 'margin': '0', 'fontSize': '0.85rem',
                        'fontFamily': 'Rajdhani, monospace'
                    })
                ])
            ], style={
                'display': 'flex', 'alignItems': 'center', 'padding': '15px',
                'backgroundColor': COLORS['sidebar_bg'], 'borderRadius': '10px',
                'marginBottom': '10px', 'border': f'1px solid {COLORS["success"]}40'
            }),
            
            html.Div([
                html.I(className="fas fa-birthday-cake", style={
                    'fontSize': '1.5rem', 'color': COLORS['warning'],
                    'marginRight': '12px'
                }),
                html.Div([
                    html.H5(f"{avg_age:.1f} yrs", style={
                        'color': COLORS['light'], 'margin': '0', 'fontWeight': '700',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    html.P("Avg Age", style={
                        'color': COLORS['warning'], 'margin': '0', 'fontSize': '0.85rem',
                        'fontFamily': 'Rajdhani, monospace'
                    })
                ])
            ], style={
                'display': 'flex', 'alignItems': 'center', 'padding': '15px',
                'backgroundColor': COLORS['sidebar_bg'], 'borderRadius': '10px',
                'marginBottom': '10px', 'border': f'1px solid {COLORS["warning"]}40'
            }),
            
            html.Div([
                html.I(className="fas fa-globe", style={
                    'fontSize': '1.5rem', 'color': COLORS['pink'],
                    'marginRight': '12px'
                }),
                html.Div([
                    html.H5(f"{citizenship_counts.index[0] if len(citizenship_counts) > 0 else 'N/A'}", style={
                        'color': COLORS['light'], 'margin': '0', 'fontWeight': '700',
                        'fontFamily': 'Orbitron, monospace', 'fontSize': '0.9rem'
                    }),
                    html.P(f"{citizenship_counts.iloc[0] if len(citizenship_counts) > 0 else 0} students", style={
                        'color': COLORS['pink'], 'margin': '0', 'fontSize': '0.85rem',
                        'fontFamily': 'Rajdhani, monospace'
                    })
                ])
            ], style={
                'display': 'flex', 'alignItems': 'center', 'padding': '15px',
                'backgroundColor': COLORS['sidebar_bg'], 'borderRadius': '10px',
                'border': f'1px solid {COLORS["pink"]}40'
            })
        ])
    ])

# Update Student Data Table - use filtered data instead of search
@app.callback(
    Output('student-data-table', 'children'),
    Input('filtered-data', 'data')
)
def update_student_table(filtered_data):
    if not filtered_data:
        return html.Div([
            html.P("🔍 No students match the current filters", 
                   style={
                       'textAlign': 'center', 
                       'color': COLORS['text_dim'], 
                       'fontSize': '1.1rem',
                       'fontFamily': 'Rajdhani, monospace',
                       'fontStyle': 'italic',
                       'marginTop': '40px'
                   })
        ])
    
    # Use filtered data
    student_data = pd.DataFrame(filtered_data)
    
    if student_data.empty:
        return html.Div([
            html.P("🔍 No students found with current filters", 
                   style={
                       'textAlign': 'center', 
                       'color': COLORS['danger'], 
                       'fontSize': '1.1rem',
                       'fontFamily': 'Rajdhani, monospace',
                       'marginTop': '40px'
                   })
        ])
    
    # Create table rows for filtered students (limit to 10 for performance)
    table_rows = []
    for _, student in student_data.head(10).iterrows():
        table_rows.append(
            html.Tr([
                html.Td(student['STUDENT ID'], style={'color': COLORS['neon_blue'], 'fontWeight': 'bold', 'fontSize': '0.8rem'}),
                html.Td(student['GENDER'], style={'color': COLORS['light'], 'fontSize': '0.8rem'}),
                html.Td(student['CITIZENSHIP'], style={'color': COLORS['neon_purple'], 'fontSize': '0.8rem'}),
                html.Td(student['DEPARTMENT'], style={'color': COLORS['accent'], 'fontSize': '0.8rem'}),
                html.Td(f"{student['CUMULATIVE_GPA']:.2f}", style={'color': COLORS['warning'], 'fontWeight': 'bold', 'fontSize': '0.8rem'}),
                html.Td(student['COURSE FUNDING'], style={'color': COLORS['text'], 'fontSize': '0.8rem'})
            ], style={
                'borderBottom': f'1px solid {COLORS["neon_blue"]}40',
                'transition': 'all 0.3s ease'
            })
        )
    
    display_count = min(10, len(student_data))
    total_count = len(student_data)
    
    return html.Div([
        html.P(f"📊 Showing {display_count} of {total_count} filtered students", 
               style={
                   'color': COLORS['neon_green'], 
                   'fontSize': '0.9rem',
                   'fontFamily': 'Orbitron, monospace',
                   'marginBottom': '15px',
                   'textAlign': 'center'
               }),
        html.Table([
            html.Thead([
                html.Tr([
                    html.Th("ID", style={'color': COLORS['neon_blue'], 'fontFamily': 'Orbitron, monospace', 'fontSize': '0.8rem'}),
                    html.Th("Gender", style={'color': COLORS['light'], 'fontFamily': 'Orbitron, monospace', 'fontSize': '0.8rem'}),
                    html.Th("Citizenship", style={'color': COLORS['neon_purple'], 'fontFamily': 'Orbitron, monospace', 'fontSize': '0.8rem'}),
                    html.Th("Department", style={'color': COLORS['accent'], 'fontFamily': 'Orbitron, monospace', 'fontSize': '0.8rem'}),
                    html.Th("GPA", style={'color': COLORS['warning'], 'fontFamily': 'Orbitron, monospace', 'fontSize': '0.8rem'}),
                    html.Th("Funding", style={'color': COLORS['text'], 'fontFamily': 'Orbitron, monospace', 'fontSize': '0.8rem'})
                ], style={
                    'borderBottom': f'2px solid {COLORS["neon_blue"]}',
                    'backgroundColor': COLORS['sidebar_bg']
                })
            ]),
            html.Tbody(table_rows)
        ], style={
            'width': '100%',
            'fontFamily': 'Rajdhani, monospace',
            'fontSize': '0.85rem',
            'borderCollapse': 'collapse'
        })
    ])

# Update chart callbacks to use filtered data
@app.callback(
    Output('gpa-by-dept-chart', 'figure'),
    Input('filtered-data', 'data')
)
def update_gpa_dept_chart(filtered_data):
    """Interactive Treemap Chart with built-in hierarchical filters"""
    
    # Use filtered data
    if not filtered_data:
        return go.Figure().add_annotation(text="No data to display", showarrow=False)
    
    df = pd.DataFrame(filtered_data)
    # Prepare hierarchical data for treemap
    treemap_data = df.groupby(['CITIZENSHIP', 'GENDER', 'COURSE FUNDING']).size().reset_index(name='COUNT')
    treemap_data['PERCENTAGE'] = (treemap_data['COUNT'] / len(df) * 100).round(1)
    
    fig = px.treemap(
        treemap_data,
        path=['CITIZENSHIP', 'GENDER', 'COURSE FUNDING'],
        values='COUNT',
        title='<b>Student Population Breakdown</b><br><sub>Click to drill down • Hover for details</sub>',
        color='COUNT',
        color_continuous_scale='RdYlBu_r',
        hover_data={'PERCENTAGE': ':.1f%'},
        height=800
    )
    
    # Apply styling
    fig.update_layout(
        title_font_size=18,
        title_x=0.5,
        font=dict(family='Orbitron, monospace', size=12),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font_color=COLORS['light'],
        title_font_color=COLORS['secondary']
    )
    
    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>' +
                     'Count: %{value}<br>' +
                     'Percentage: %{customdata[0]:.1f}%<br>' +
                     '<extra></extra>',
        textinfo='label+percent parent',
        textfont_size=11
    )
    
    return fig

@app.callback(
    Output('citizenship-chart', 'figure'),
    Input('filtered-data', 'data')
)
def update_citizenship_chart(filtered_data):
    """Interactive GPA vs Age Scatter with built-in dropdown filters"""
    
    # Use filtered data
    if not filtered_data:
        return go.Figure().add_annotation(text="No data to display", showarrow=False)
    
    df = pd.DataFrame(filtered_data)
    scatter_data = df.dropna(subset=['CUMULATIVE_GPA', 'AGE_AT_QUALIFICATION']).copy()
    if 'SUCCESS_SCORE' not in scatter_data.columns:
        scatter_data['SUCCESS_SCORE'] = (scatter_data['CUMULATIVE_GPA'] / 4.0 * 100).round(1)
    
    # Create figure
    fig = go.Figure()
    
    citizenship_types = ['Citizen', 'PR', 'International']
    citizenship_colors = [COLORS['primary'], COLORS['secondary'], COLORS['pink']]
    
    # Add traces for each citizenship type
    for i, citizenship in enumerate(citizenship_types):
        citizenship_subset = scatter_data[scatter_data['CITIZENSHIP'] == citizenship]
        if not citizenship_subset.empty:
            fig.add_trace(go.Scatter(
                x=citizenship_subset['AGE_AT_QUALIFICATION'],
                y=citizenship_subset['CUMULATIVE_GPA'],
                mode='markers',
                marker=dict(
                    size=10,
                    color=citizenship_subset['SUCCESS_SCORE'],
                    colorscale='Viridis',
                    showscale=i==0,
                    line=dict(width=2, color='white'),
                    opacity=0.8,
                    colorbar=dict(title="Success Score", len=0.8) if i==0 else None
                ),
                text=[f"Student ID: {row['STUDENT ID']}<br>" +
                      f"GPA: {row['CUMULATIVE_GPA']:.2f}<br>" +
                      f"Age: {row['AGE_AT_QUALIFICATION']:.0f} years<br>" +
                      f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
                      f"Citizenship: {row['CITIZENSHIP']}"
                      for _, row in citizenship_subset.iterrows()],
                hovertemplate='%{text}<extra></extra>',
                name=f'{citizenship} ({len(citizenship_subset)} students)',
                visible=True
            ))
    
    # Add high achievers traces (GPA > 3.5)
    for i, citizenship in enumerate(citizenship_types):
        high_achievers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                     (scatter_data['CUMULATIVE_GPA'] > 3.5)]
        if not high_achievers.empty:
            fig.add_trace(go.Scatter(
                x=high_achievers['AGE_AT_QUALIFICATION'],
                y=high_achievers['CUMULATIVE_GPA'],
                mode='markers',
                marker=dict(
                    size=12,
                    color=citizenship_colors[i],
                    line=dict(width=3, color='gold'),
                    opacity=0.9,
                    symbol='star'
                ),
                name=f'{citizenship} High Achievers ({len(high_achievers)})',
                visible=False
            ))
    
    # Create dropdown options
    num_citizenship = len(citizenship_types)
    dropdown_buttons = [
        dict(label='🌍 All Citizenship Types', 
             method='update', 
             args=[{'visible': [True] * num_citizenship + [False] * num_citizenship}]),
        dict(label='⭐ High Achievers (GPA > 3.5)', 
             method='update', 
             args=[{'visible': [False] * num_citizenship + [True] * num_citizenship}])
    ]
    
    # Add individual citizenship options
    for i, citizenship in enumerate(citizenship_types):
        visibility = [False] * (num_citizenship * 2)
        visibility[i] = True
        dropdown_buttons.append(
            dict(label=f'🎯 {citizenship} Only',
                 method='update',
                 args=[{'visible': visibility}])
        )
    
    # Configure layout with embedded dropdown
    fig.update_layout(
        title=dict(
            text='<b>🎯 Interactive Age vs GPA Analysis</b><br>' +
                 '<sub>Use filters below to explore different groups</sub>',
            font=dict(size=18, family='Orbitron, monospace', color=COLORS['secondary']),
            x=0.5
        ),
        xaxis=dict(title='Age at Qualification', gridcolor='rgba(200,200,200,0.3)'),
        yaxis=dict(title='Cumulative GPA', gridcolor='rgba(200,200,200,0.3)', range=[1.0, 4.0]),
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        height=700,
        margin=dict(t=150, l=80, r=120, b=80),
        showlegend=True,
        legend=dict(yanchor="top", y=0.98, xanchor="left", x=1.02),
        updatemenus=[
            dict(
                buttons=dropdown_buttons,
                direction="down",
                showactive=True,
                x=0.02, xanchor="left",
                y=1.12, yanchor="top",
                bgcolor=COLORS['card_bg'],
                bordercolor=COLORS['neon_blue'],
                borderwidth=2
            )
        ],
        annotations=[
            dict(
                text="🔍 Filter by Group:",
                x=0.02, y=1.18,
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_blue'])
            )
        ]
    )
    
    # Add reference lines
    fig.add_hline(y=2.0, line_dash="dot", line_color=COLORS['danger'], line_width=2)
    fig.add_hline(y=3.5, line_dash="dot", line_color=COLORS['success'], line_width=2)
    
    return fig

@app.callback(
    Output('gpa-age-scatter', 'figure'),
    Input('interval-update', 'n_intervals')
)
def update_scatter_chart(n):
    """Animated GPA Trends with built-in play/pause controls"""
    
    # Create period data from results
    gpa_trends = df_results.groupby(['PERIOD', 'STUDENT ID']).agg({'GPA': 'mean'}).reset_index()
    gpa_trends = gpa_trends.merge(df_students[['STUDENT ID', 'CITIZENSHIP']], on='STUDENT ID')
    
    period_summary = gpa_trends.groupby(['PERIOD', 'CITIZENSHIP']).agg({
        'GPA': ['mean', 'count']
    }).round(2)
    period_summary.columns = ['AVG_GPA', 'STUDENT_COUNT']
    period_summary = period_summary.reset_index()
    
    periods = sorted(period_summary['PERIOD'].unique()) if not period_summary.empty else ['2023 Sem 1']
    citizenship_types = ['Citizen', 'PR', 'International']
    citizenship_colors = [COLORS['primary'], COLORS['secondary'], COLORS['pink']]
    
    # Create frames for animation
    frames = []
    for period in periods:
        period_frame = period_summary[period_summary['PERIOD'] == period]
        
        if not period_frame.empty:
            frames.append(
                go.Frame(
                    data=[
                        go.Scatter(
                            x=period_frame['CITIZENSHIP'],
                            y=period_frame['AVG_GPA'],
                            mode='markers+lines+text',
                            marker=dict(
                                size=[15 + (count/3) for count in period_frame['STUDENT_COUNT']],
                                color=citizenship_colors[:len(period_frame)],
                                line=dict(width=3, color='white'),
                                opacity=0.8
                            ),
                            line=dict(width=4, color=COLORS['secondary']),
                            text=[f"{gpa:.2f}" for gpa in period_frame['AVG_GPA']],
                            textposition='top center',
                            textfont=dict(size=14, color=COLORS['light']),
                            hovertemplate='<b>%{x}</b><br>GPA: %{y:.2f}<br>Students: %{customdata}<br><extra></extra>',
                            customdata=period_frame['STUDENT_COUNT'],
                            name=period
                        )
                    ],
                    name=period
                )
            )
    
    # Create initial frame
    if periods:
        initial_data = period_summary[period_summary['PERIOD'] == periods[0]]
    else:
        initial_data = pd.DataFrame({
            'CITIZENSHIP': citizenship_types,
            'AVG_GPA': [2.5, 2.7, 2.8],
            'STUDENT_COUNT': [50, 30, 20]
        })
    
    fig = go.Figure(
        data=[
            go.Scatter(
                x=initial_data['CITIZENSHIP'],
                y=initial_data['AVG_GPA'],
                mode='markers+lines+text',
                marker=dict(
                    size=[15 + (count/3) for count in initial_data['STUDENT_COUNT']],
                    color=citizenship_colors[:len(initial_data)],
                    line=dict(width=3, color='white'),
                    opacity=0.8
                ),
                line=dict(width=4, color=COLORS['secondary']),
                text=[f"{gpa:.2f}" for gpa in initial_data['AVG_GPA']],
                textposition='top center',
                textfont=dict(size=14, color=COLORS['light']),
                hovertemplate='<b>%{x}</b><br>GPA: %{y:.2f}<br>Students: %{customdata}<br><extra></extra>',
                customdata=initial_data['STUDENT_COUNT']
            )
        ],
        frames=frames if frames else []
    )
    
    # Enhanced layout with animation controls
    fig.update_layout(
        title=dict(
            text='<b>📈 GPA Evolution Over Time</b><br><sub>Use controls below to play animation</sub>',
            font=dict(size=18, family='Orbitron, monospace', color=COLORS['accent']),
            x=0.5
        ),
        xaxis=dict(title='Citizenship Status', gridcolor='rgba(200,200,200,0.3)'),
        yaxis=dict(title='Average GPA', range=[1.5, 4.0], gridcolor='rgba(200,200,200,0.3)'),
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        height=800,
        showlegend=False,
        updatemenus=[
            dict(
                type='buttons',
                showactive=False,
                x=0.1, xanchor='left',
                y=1.18, yanchor='top',
                bgcolor=COLORS['card_bg'],
                bordercolor=COLORS['accent'],
                borderwidth=2,
                buttons=[
                    dict(label='▶️ Play',
                         method='animate',
                         args=[None, {'frame': {'duration': 1200, 'redraw': True},
                                     'fromcurrent': True,
                                     'transition': {'duration': 400}}]),
                    dict(label='⏸️ Pause',
                         method='animate',
                         args=[[None], {'frame': {'duration': 0, 'redraw': False},
                                       'mode': 'immediate'}]),
                    dict(label='🔄 Reset',
                         method='animate',
                         args=[[periods[0] if periods else None], 
                               {'frame': {'duration': 0, 'redraw': True}}])
                ]
            )
        ] if frames else []
    )
    
    return fig


def update_gpa_trend_chart(filtered_data):
    """GPA Trend Over Time Chart with Filtered Data"""
    
    # Use filtered data
    if not filtered_data:
        return go.Figure().add_annotation(
            text="No data to display", 
            showarrow=False,
            font=dict(size=16, color=COLORS['light'])
        )
    
    df = pd.DataFrame(filtered_data)
    
    # Create trend data by commencement date
    if 'COMMENCEMENT DATE' in df.columns:
        df['COMMENCEMENT DATE'] = pd.to_datetime(df['COMMENCEMENT DATE'], errors='coerce')
        df['Year'] = df['COMMENCEMENT DATE'].dt.year
        
        # Group by year and calculate average GPA
        trend_data = df.groupby('Year')['CUMULATIVE_GPA'].agg(['mean', 'count']).reset_index()
        trend_data.columns = ['Year', 'Avg_GPA', 'Student_Count']
        
        # Create line chart
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=trend_data['Year'],
            y=trend_data['Avg_GPA'],
            mode='lines+markers',
            name='Average GPA',
            line=dict(color=COLORS['neon_green'], width=3),
            marker=dict(size=8, color=COLORS['neon_blue']),
            hovertemplate='<b>Year: %{x}</b><br>' +
                         'Average GPA: %{y:.2f}<br>' +
                         'Students: %{customdata}<br>' +
                         '<extra></extra>',
            customdata=trend_data['Student_Count']
        ))
        
        fig.update_layout(
            title='<b>GPA Trends Over Time</b><br><sub>Average cumulative GPA by commencement year</sub>',
            title_font_size=18,
            title_x=0.5,
            xaxis_title='Commencement Year',
            yaxis_title='Average GPA',
            font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            height=600,
            yaxis=dict(range=[0, 4]),
            xaxis=dict(tickmode='linear')
        )
        
        return fig
    else:
        return go.Figure().add_annotation(
            text="Date information not available", 
            showarrow=False,
            font=dict(size=16, color=COLORS['light'])
        )

# ================================
# RUN APPLICATION
# ================================
if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=8051)

📊 Generating mock data with correct column structure...


### LEE SHEN LEI

# PART 1: DATA LOADING & PREPROCESSING
## Setup, Data Loading, and Calculated Fields

This section handles all data operations, imports, and derived field calculations needed for the analysis.

In [21]:
# === IMPORTS & SETUP ===
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# === AESTHETIC CONFIGURATION ===
# FUTURISTIC GLOWING COLOR SCHEME (matching dashboard)
COLORS = {
    'primary': '#8B5FBF',      # Purple gradient
    'secondary': '#22D3EE',    # Cyan blue  
    'accent': '#F97316',       # Orange accent
    'success': '#10B981',      # Green
    'warning': '#F59E0B',      # Amber
    'danger': '#EF4444',       # Red
    'pink': '#EC4899',         # Pink for cards
    'dark_bg': '#0A0A1A',      # Ultra dark background
    'card_bg': '#1A1A2E',      # Dark card background with glow
    'sidebar_bg': '#16213E',   # Glowing sidebar
    'light': '#FFFFFF',        # White
    'text': '#E2E8F0',         # Light gray text
    'text_dim': '#94A3B8',     # Dimmed text
    'neon_blue': '#00F5FF',    # Neon blue glow
    'neon_purple': '#BF40BF',  # Neon purple glow
    'neon_green': '#39FF14',   # Neon green glow
    'glow_shadow': 'rgba(34, 211, 238, 0.4)',  # Glowing shadow
    'neutral': '#6C7B7F'       # Cool gray
}

# Professional color palettes using dashboard colors
citizenship_colors = [COLORS['primary'], COLORS['secondary'], COLORS['accent'], COLORS['pink']]
funding_colors = [COLORS['neon_blue'], COLORS['neon_purple'], COLORS['neon_green'], COLORS['accent'], COLORS['pink']]
gender_colors = [COLORS['primary'], COLORS['secondary']]

# Global styling with futuristic dark theme
template_config = {
    'layout': {
        'font': {'family': 'Orbitron, monospace', 'size': 12, 'color': COLORS['light']},
        'paper_bgcolor': COLORS['dark_bg'],
        'plot_bgcolor': 'rgba(0,0,0,0)',
        'margin': {'l': 80, 'r': 120, 't': 150, 'b': 80},
        'hovermode': 'closest',
        'showlegend': True,
        'legend': {
            'orientation': 'h',
            'yanchor': 'bottom',
            'y': -0.2,
            'xanchor': 'center',
            'x': 0.5,
            'bgcolor': 'rgba(255,255,255,0.8)',
            'bordercolor': COLORS['neutral'],
            'borderwidth': 1
        }
    }
}

print("📊 Imports and aesthetic configuration completed!")

📊 Imports and aesthetic configuration completed!


In [22]:
# === DATA LOADING & PREPROCESSING ===
print("📊 Loading and preprocessing data...")

# Load datasets
try:
    profiles = pd.read_excel("Cleaned_Student_Profiles.xlsx")
    results = pd.read_excel("Cleaned_Semester_Results.xlsx")
    courses = pd.read_excel("Cleaned_Course_Codes.xlsx")
    print("✅ Data loaded successfully")
except FileNotFoundError as e:
    print(f"❌ Error loading data: {e}")
    # Create mock data for demonstration
    print("🔄 Creating mock data for demonstration...")
    np.random.seed(42)
    n_students = 500
    
    profiles = pd.DataFrame({
        'STUDENT ID': [f'STU{i:05d}' for i in range(1, n_students + 1)],
        'GENDER': np.random.choice(['Male', 'Female'], n_students),
        'CITIZENSHIP': np.random.choice(['Citizen', 'PR', 'International'], n_students, p=[0.6, 0.25, 0.15]),
        'HIGHEST QUALIFICATION': np.random.choice(['Diploma', 'Bachelor', 'Master', 'PhD'], n_students, p=[0.3, 0.4, 0.25, 0.05]),
        'COURSE FUNDING': np.random.choice(['Self-Funded', 'Scholarship', 'Sponsored', 'Loan'], n_students),
        'COMMENCEMENT DATE': pd.date_range('2020-01-01', '2023-12-31', periods=n_students),
        'COMPLETION DATE': pd.date_range('2022-01-01', '2026-12-31', periods=n_students),
        'FULL-TIME OR PART-TIME': np.random.choice(['Full-Time', 'Part-Time'], n_students, p=[0.7, 0.3])
    })
    
    # Generate results data
    periods = ['2020 Sem 1', '2020 Sem 2', '2021 Sem 1', '2021 Sem 2', '2022 Sem 1', '2022 Sem 2', '2023 Sem 1', '2023 Sem 2']
    n_results = n_students * 3  # Average 3 semesters per student
    results = pd.DataFrame({
        'STUDENT ID': np.random.choice(profiles['STUDENT ID'], n_results),
        'PERIOD': np.random.choice(periods, n_results),
        'GPA': np.random.beta(2, 1, n_results) * 3 + 1  # GPA between 1-4
    })
    results['GPA'] = results['GPA'].round(2)

# Parse dates
date_columns = ['COMMENCEMENT DATE', 'COMPLETION DATE']
for col in date_columns:
    if col in profiles.columns:
        profiles[col] = pd.to_datetime(profiles[col], errors='coerce')

print("✅ Data loading completed!")

📊 Loading and preprocessing data...
✅ Data loaded successfully
✅ Data loading completed!


In [23]:
# === CALCULATE DERIVED METRICS ===
print("🔧 Calculating derived metrics...")

# 1. Cumulative GPA per student
cum_gpa = results.groupby('STUDENT ID', as_index=False)['GPA'].agg({
    'CUMULATIVE_GPA': 'mean',
    'TOTAL_SEMESTERS': 'count',
    'BEST_GPA': 'max',
    'WORST_GPA': 'min'
}).round(2)

# 2. Merge with profiles
df = profiles.merge(cum_gpa, on='STUDENT ID', how='left')

# 3. Calculate completion time and academic outcome
df['COMPLETION_MONTHS'] = (df['COMPLETION DATE'] - df['COMMENCEMENT DATE']).dt.days / 30.44
df['ACADEMIC_OUTCOME'] = np.where(df['CUMULATIVE_GPA'] >= 2.0, 'Pass', 'Fail')
df['PERFORMANCE_TIER'] = pd.cut(df['CUMULATIVE_GPA'], 
                               bins=[0, 2.0, 2.5, 3.0, 4.0], 
                               labels=['Poor', 'Average', 'Good', 'Excellent'])

# 4. NEW CALCULATED FIELDS - Advanced Analytics
print("🔧 Creating advanced calculated fields...")

# Academic Progress Metrics
df['GPA_IMPROVEMENT'] = df['BEST_GPA'] - df['WORST_GPA']  # Range of performance
df['GPA_VOLATILITY'] = df['GPA_IMPROVEMENT'] / df['TOTAL_SEMESTERS']  # Volatility per semester
df['ACADEMIC_EFFICIENCY'] = df['CUMULATIVE_GPA'] / df['COMPLETION_MONTHS']  # Performance per month
df['STUDY_INTENSITY'] = df['TOTAL_SEMESTERS'] / df['COMPLETION_MONTHS'] * 12  # Semesters per year

# Performance Categories
df['FAST_TRACK'] = np.where(df['COMPLETION_MONTHS'] <= 24, 'Fast Track', 
                   np.where(df['COMPLETION_MONTHS'] <= 36, 'Standard', 'Extended'))

# Financial & Demographic Scoring
funding_priority = {'Scholarship': 4, 'Sponsored': 3, 'Loan': 2, 'Self-Funded': 1}
df['FUNDING_PRIORITY'] = df['COURSE FUNDING'].map(funding_priority)

# Academic Risk Assessment
df['RISK_SCORE'] = (
    np.where(df['CUMULATIVE_GPA'] < 2.0, 3, 0) +  # Academic risk
    np.where(df['COMPLETION_MONTHS'] > 48, 2, 0) +  # Time risk
    np.where(df['GPA_VOLATILITY'] > 0.1, 1, 0)     # Consistency risk
)
df['RISK_CATEGORY'] = pd.cut(df['RISK_SCORE'], 
                            bins=[-1, 0, 2, 6], 
                            labels=['Low Risk', 'Medium Risk', 'High Risk'])

# Success Prediction Score (0-100)
df['SUCCESS_SCORE'] = (
    (df['CUMULATIVE_GPA'] / 4.0 * 40) +  # 40% weight on GPA
    (np.where(df['COMPLETION_MONTHS'] <= 36, 30, 
             np.where(df['COMPLETION_MONTHS'] <= 48, 20, 10))) +  # 30% on completion speed
    (df['FUNDING_PRIORITY'] * 5) +  # 20% on funding
    (np.where(df['GPA_VOLATILITY'] <= 0.05, 10, 
             np.where(df['GPA_VOLATILITY'] <= 0.1, 5, 0)))  # 10% on consistency
).round(1)

# 5. Prepare period-wise data for time series analysis
period_data = results.merge(profiles[['STUDENT ID', 'CITIZENSHIP', 'GENDER', 'COURSE FUNDING']], on='STUDENT ID')
period_summary = period_data.groupby(['PERIOD', 'CITIZENSHIP'], as_index=False)['GPA'].mean().round(2)

print(f"📈 Dataset Summary: {len(df)} students, {len(results)} semester records")
print("✅ All calculated fields completed!")
print("=" * 60)

🔧 Calculating derived metrics...
🔧 Creating advanced calculated fields...
📈 Dataset Summary: 285 students, 531 semester records
✅ All calculated fields completed!


# PART 2: INDIVIDUAL CHART CREATIONS
## Professional Analytics Charts with Template Aesthetics

This section creates 4 individual charts with modern styling, interactive elements, and comprehensive insights matching the dashboard template aesthetic.

In [24]:
# === CHART 1: PLOTLY EXPRESS - Student Distribution Treemap ===
print("🎨 Creating Chart 1: Student Distribution Treemap...")

# Prepare hierarchical data for treemap
treemap_data = df.groupby(['CITIZENSHIP', 'GENDER', 'COURSE FUNDING']).size().reset_index(name='COUNT')
treemap_data['PERCENTAGE'] = (treemap_data['COUNT'] / len(df) * 100).round(1)

fig1 = px.treemap(
    treemap_data,
    path=['CITIZENSHIP', 'GENDER', 'COURSE FUNDING'],
    values='COUNT',
    title='<b>🌍 Student Population Breakdown</b><br><sub>Distribution by Citizenship, Gender, and Funding Type • Click to drill down</sub>',
    color='COUNT',
    color_continuous_scale='Viridis',
    hover_data={'PERCENTAGE': ':.1f%'}
)

fig1.update_layout(
    font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
    title_font_size=18,
    title_x=0.5,
    title_font_color=COLORS['light'],
    paper_bgcolor=COLORS['dark_bg'],
    height=700,
    margin=dict(t=150, l=20, r=20, b=20)
)

fig1.update_traces(
    textinfo='label+value+percent parent',
    textfont_size=11,
    textfont_color=COLORS['light'],
    hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{customdata[0]:.1f}%<extra></extra>'
)

fig1.show()
print("✅ Chart 1 completed!")

🎨 Creating Chart 1: Student Distribution Treemap...


✅ Chart 1 completed!


In [25]:
# === CHART 2: INTERACTIVE GPA vs COMPLETION TIME ANALYSIS ===
print("🎨 Creating Chart 2: Interactive GPA vs Completion Time Analysis...")

# Prepare data for scatter plot analysis
scatter_data = df.dropna(subset=['CUMULATIVE_GPA', 'COMPLETION_MONTHS', 'SUCCESS_SCORE'])

# Create interactive scatter plot with multiple filtering options
fig2 = go.Figure()

# Create traces for each citizenship type (initially all visible)
citizenship_types = scatter_data['CITIZENSHIP'].unique()
for i, citizenship in enumerate(citizenship_types):
    citizenship_subset = scatter_data[scatter_data['CITIZENSHIP'] == citizenship]
    
    fig2.add_trace(go.Scatter(
        x=citizenship_subset['COMPLETION_MONTHS'],
        y=citizenship_subset['CUMULATIVE_GPA'],
        mode='markers',
        marker=dict(
            size=12 + (citizenship_subset['RISK_SCORE'] * 4),  # Size by risk score
            color=citizenship_subset['SUCCESS_SCORE'],
            colorscale='Viridis',
            showscale=True if i == 0 else False,  # Show colorbar only once
            colorbar=dict(
                title="Success<br>Score<br>(0-100)",
                titlefont=dict(size=12, family='Arial Black'),
                tickfont=dict(size=10),
                len=0.8
            ),
            line=dict(width=2, color='white'),
            opacity=0.8,
            symbol='circle'
        ),
        text=[f"Student ID: {row['STUDENT ID']}<br>" +
              f"GPA: {row['CUMULATIVE_GPA']:.2f}<br>" +
              f"Completion: {row['COMPLETION_MONTHS']:.1f} months<br>" +
              f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
              f"Risk Category: {row['RISK_CATEGORY']}<br>" +
              f"Citizenship: {row['CITIZENSHIP']}<br>" +
              f"Gender: {row['GENDER']}<br>" +
              f"Funding: {row['COURSE FUNDING']}"
              for _, row in citizenship_subset.iterrows()],
        hovertemplate='%{text}<extra></extra>',
        name=f'{citizenship} ({len(citizenship_subset)} students)',
        visible=True
    ))

# Add HIGH ACHIEVERS traces (GPA > 3.5 AND completion < 5 months) for each citizenship
for i, citizenship in enumerate(citizenship_types):
    high_achievers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                 (scatter_data['CUMULATIVE_GPA'] > 3.5) &
                                 (scatter_data['COMPLETION_MONTHS'] < 5)]
    
    fig2.add_trace(go.Scatter(
        x=high_achievers['COMPLETION_MONTHS'],
        y=high_achievers['CUMULATIVE_GPA'],
        mode='markers',
        marker=dict(
            size=12 + (high_achievers['RISK_SCORE'] * 4),
            color=high_achievers['SUCCESS_SCORE'],
            colorscale='Viridis',
            showscale=False,
            line=dict(width=2, color='white'),
            opacity=0.8,
            symbol='circle'
        ),
        text=[f"Student ID: {row['STUDENT ID']}<br>" +
              f"GPA: {row['CUMULATIVE_GPA']:.2f} ⭐⭐<br>" +
              f"Completion: {row['COMPLETION_MONTHS']:.1f} months 🚀<br>" +
              f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
              f"Risk Category: {row['RISK_CATEGORY']}<br>" +
              f"Citizenship: {row['CITIZENSHIP']}<br>" +
              f"Gender: {row['GENDER']}<br>" +
              f"Funding: {row['COURSE FUNDING']}"
              for _, row in high_achievers.iterrows()],
        hovertemplate='%{text}<extra></extra>',
        name=f'{citizenship} High Achievers ({len(high_achievers)} students)',
        visible=False
    ))

# Add UNDERPERFORMERS traces (GPA < 2.5) for each citizenship
for i, citizenship in enumerate(citizenship_types):
    underperformers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                  (scatter_data['CUMULATIVE_GPA'] < 2.5)]
    
    fig2.add_trace(go.Scatter(
        x=underperformers['COMPLETION_MONTHS'],
        y=underperformers['CUMULATIVE_GPA'],
        mode='markers',
        marker=dict(
            size=12 + (underperformers['RISK_SCORE'] * 4),
            color=underperformers['SUCCESS_SCORE'],
            colorscale='Viridis',
            showscale=False,
            line=dict(width=2, color='white'),
            opacity=0.8,
            symbol='circle'
        ),
        text=[f"Student ID: {row['STUDENT ID']}<br>" +
              f"GPA: {row['CUMULATIVE_GPA']:.2f} 🚨<br>" +
              f"Completion: {row['COMPLETION_MONTHS']:.1f} months<br>" +
              f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
              f"Risk Category: {row['RISK_CATEGORY']}<br>" +
              f"Citizenship: {row['CITIZENSHIP']}<br>" +
              f"Gender: {row['GENDER']}<br>" +
              f"Funding: {row['COURSE FUNDING']}"
              for _, row in underperformers.iterrows()],
        hovertemplate='%{text}<extra></extra>',
        name=f'{citizenship} Underperformers ({len(underperformers)} students)',
        visible=False
    ))

# Create dropdown options for different views
num_citizenship = len(citizenship_types)
dropdown_buttons = [
    # Show all citizenship types (regular view)
    dict(label='🌍 All Citizenship Types', 
         method='update', 
         args=[{'visible': [True] * num_citizenship + [False] * (num_citizenship * 2)}]),
]

# Add individual citizenship options (regular view)
for i, citizenship in enumerate(citizenship_types):
    visibility = [False] * (num_citizenship * 3)  # Total traces = citizenship + high achievers + underperformers
    visibility[i] = True  # Show only this citizenship's regular trace
    dropdown_buttons.append(
        dict(label=f'🎯 {citizenship} Only',
             method='update',
             args=[{'visible': visibility}])
    )

# Add high achievers view (show all high achiever traces)
high_achiever_visibility = [False] * num_citizenship + [True] * num_citizenship + [False] * num_citizenship
dropdown_buttons.append(
    dict(label='⭐ High Achievers (GPA > 3.5 & < 5 months)', 
         method='update', 
         args=[{'visible': high_achiever_visibility}])
)

# Add underperformers view (show all underperformer traces)
underperformer_visibility = [False] * (num_citizenship * 2) + [True] * num_citizenship
dropdown_buttons.append(
    dict(label='🚨 Underperformers (GPA < 2.5)', 
         method='update', 
         args=[{'visible': underperformer_visibility}])
)

# Configure layout with enhanced interactivity and proper filter positioning
fig2.update_layout(
    title=dict(
        text='<b>🎯 Interactive GPA vs Completion Time Analysis</b><br>' +
             '<sub>Success score by color • Risk level by marker size • Use filters to explore different groups</sub>',
        font=dict(size=20, family='Orbitron, monospace', color=COLORS['light']),
        x=0.5,
        y=0.95
    ),
    xaxis=dict(
        title=dict(text='Completion Time (Months)', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_blue'])),
        tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
        gridcolor='rgba(200,200,200,0.2)',
        gridwidth=1,
        range=[0, max(scatter_data['COMPLETION_MONTHS']) + 5]
    ),
    yaxis=dict(
        title=dict(text='Cumulative GPA', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_purple'])),
        tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
        gridcolor='rgba(200,200,200,0.2)',
        gridwidth=1,
        range=[1.0, 4.0]
    ),
    font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
    paper_bgcolor=COLORS['dark_bg'],
    plot_bgcolor='rgba(0,0,0,0)',
    height=700,
    margin=dict(t=150, l=80, r=120, b=80),
    showlegend=True,
    legend=dict(
        orientation='v',
        yanchor='top',
        y=0.98,
        xanchor='left',
        x=1.02,
        bgcolor=COLORS['card_bg'],
        bordercolor=COLORS['neon_blue'],
        borderwidth=1,
        font=dict(color=COLORS['light'])
    ),
    
    # Add interactive dropdown menu with proper positioning
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
            x=0.02,
            xanchor="left",
            y=1.12,
            yanchor="top",
            bgcolor=COLORS['card_bg'],
            bordercolor=COLORS['neon_green'],
            borderwidth=2,
            font=dict(color=COLORS['light'])
        )
    ],
    
    # Add filter label with proper positioning
    annotations=[
        dict(
            text="🔍 Filter by Group:",
            x=0.02, y=1.18,
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_green'])
        )
    ]
)

# Add reference lines with dashboard colors
fig2.add_hline(y=2.0, line_dash="dot", line_color=COLORS['danger'], line_width=2, opacity=0.8, 
              annotation_text="Pass Threshold (2.0)")
fig2.add_hline(y=3.5, line_dash="dot", line_color=COLORS['success'], line_width=2, opacity=0.8, 
              annotation_text="Excellence Threshold (3.5)")

# Add quadrant background shading with dashboard colors
fig2.add_shape(type="rect", xref="paper", yref="y", x0=0, y0=3.5, x1=1, y1=4.0,
               fillcolor=f"rgba({int(COLORS['success'][1:3], 16)}, {int(COLORS['success'][3:5], 16)}, {int(COLORS['success'][5:7], 16)}, 0.1)", 
               layer="below", line_width=0)
fig2.add_shape(type="rect", xref="paper", yref="y", x0=0, y0=2.0, x1=1, y1=3.5,
               fillcolor=f"rgba({int(COLORS['secondary'][1:3], 16)}, {int(COLORS['secondary'][3:5], 16)}, {int(COLORS['secondary'][5:7], 16)}, 0.1)", 
               layer="below", line_width=0)
fig2.add_shape(type="rect", xref="paper", yref="y", x0=0, y0=1.0, x1=1, y1=2.0,
               fillcolor=f"rgba({int(COLORS['danger'][1:3], 16)}, {int(COLORS['danger'][3:5], 16)}, {int(COLORS['danger'][5:7], 16)}, 0.1)", 
               layer="below", line_width=0)

fig2.show()
print("✅ Chart 2 completed!")

🎨 Creating Chart 2: Interactive GPA vs Completion Time Analysis...


✅ Chart 2 completed!


In [26]:
# === CHART 3: INTERACTIVE BOX PLOT - COMPLETION TIME ANALYSIS ===
print("🎨 Creating Chart 3: Interactive Completion Time Box Plot...")

# Filter out invalid completion times
completion_data = df.dropna(subset=['COMPLETION_MONTHS'])
completion_data = completion_data[completion_data['COMPLETION_MONTHS'] > 0]
completion_data = completion_data[completion_data['COMPLETION_MONTHS'] <= 60]  # Reasonable max of 5 years

fig3 = go.Figure()

# Add box plot for all citizenship types
for i, citizenship in enumerate(completion_data['CITIZENSHIP'].unique()):
    citizenship_data = completion_data[completion_data['CITIZENSHIP'] == citizenship]
    
    fig3.add_trace(
        go.Box(
            y=citizenship_data['COMPLETION_MONTHS'],
            name=citizenship,
            marker_color=citizenship_colors[i % len(citizenship_colors)],
            boxpoints='outliers',  # Show outliers
            jitter=0.3,  # Add some jitter to points
            pointpos=-1.8,  # Position of points
            marker=dict(
                size=4,
                opacity=0.6,
                line=dict(width=1, color='white')
            ),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Completion Time: %{y:.1f} months<br>' +
                         'Student Count: ' + str(len(citizenship_data)) + '<br>' +
                         '<extra></extra>',
            visible=True
        )
    )

# Add "All Categories" option (show all)
all_visible = [True] * len(completion_data['CITIZENSHIP'].unique())

# Create dropdown options
dropdown_buttons = [
    dict(label='All Citizenship Types',
         method='update',
         args=[{'visible': all_visible}])
]

# Add individual citizenship type options
for i, citizenship in enumerate(completion_data['CITIZENSHIP'].unique()):
    visibility = [False] * len(completion_data['CITIZENSHIP'].unique())
    visibility[i] = True
    dropdown_buttons.append(
        dict(label=f'{citizenship} Only',
             method='update',
             args=[{'visible': visibility}])
    )

fig3.update_layout(
    title='<b>🔍 Course Completion Time Distribution</b><br><sub>Interactive box plot analysis of completion duration by citizenship status</sub>',
    title_font_size=18,
    title_x=0.5,
    title_font_color=COLORS['light'],
    yaxis_title='Completion Time (Months)',
    xaxis_title='Citizenship Status',
    font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
    paper_bgcolor=COLORS['dark_bg'],
    plot_bgcolor='rgba(0,0,0,0)',
    height=700,
    margin=dict(t=150, l=80, r=80, b=80),
    showlegend=False,
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction='down',
            showactive=True,
            x=0.02,
            xanchor='left',
            y=1.12,
            yanchor='top',
            bgcolor=COLORS['card_bg'],
            bordercolor=COLORS['neon_purple'],
            borderwidth=2,
            font=dict(color=COLORS['light'])
        )
    ],
    annotations=[
        dict(
            text="🔍 Filter Options:",
            x=0.02, y=1.18,
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_purple'])
        )
    ]
)

# Add grid lines for better readability with dashboard theme
fig3.update_layout(
    yaxis=dict(
        gridcolor='rgba(200,200,200,0.2)',
        gridwidth=1,
        range=[0, 60],
        title_font_color=COLORS['neon_green'],
        tickfont=dict(color=COLORS['text'])
    ),
    xaxis=dict(
        gridcolor='rgba(200,200,200,0.2)',
        gridwidth=1,
        title_font_color=COLORS['neon_blue'],
        tickfont=dict(color=COLORS['text'])
    )
)

fig3.show()
print("✅ Chart 3 completed!")

🎨 Creating Chart 3: Interactive Completion Time Box Plot...


✅ Chart 3 completed!


In [27]:
# === CHART 4: ANIMATED GPA TRENDS OVER TIME ===
print("🎨 Creating Chart 4: Animated GPA Performance Trends...")

# Use the existing period_data which has semester GPA information
gpa_trends = period_data.groupby(['PERIOD', 'CITIZENSHIP'], as_index=False).agg({
    'GPA': ['mean', 'count']
}).round(2)

# Flatten column names
gpa_trends.columns = ['PERIOD', 'CITIZENSHIP', 'AVG_GPA', 'STUDENT_COUNT']

# Filter out periods/citizenship combinations with very few students (less than 3)
gpa_trends = gpa_trends[gpa_trends['STUDENT_COUNT'] >= 3]

# Get unique periods and citizenship types for consistent animation
periods = sorted(gpa_trends['PERIOD'].unique())
citizenship_types = sorted(gpa_trends['CITIZENSHIP'].unique())

# Create frames for animation with enhanced visuals
frames = []
for period in periods:
    period_frame = gpa_trends[gpa_trends['PERIOD'] == period]
    
    if len(period_frame) > 0:  # Only create frame if data exists
        frames.append(
            go.Frame(
                data=[
                    go.Scatter(
                        x=period_frame['CITIZENSHIP'],
                        y=period_frame['AVG_GPA'],
                        mode='markers+lines+text',
                        marker=dict(
                            size=[20 + (count/5) for count in period_frame['STUDENT_COUNT']],  # Size based on student count
                            color=citizenship_colors[:len(period_frame)],
                            line=dict(width=3, color='white'),
                            opacity=0.8,
                            symbol='circle'
                        ),
                        line=dict(
                            width=4, 
                            color=COLORS['primary'],
                            shape='spline',  # Smooth curved lines
                            smoothing=0.3
                        ),
                        text=[f"{gpa:.2f}" for gpa in period_frame['AVG_GPA']],
                        textposition='top center',
                        textfont=dict(size=14, color=COLORS['text'], family='Arial Black'),
                        hovertemplate='<b>%{x}</b><br>' +
                                    'Average GPA: %{y:.2f}<br>' +
                                    'Students: %{customdata}<br>' +
                                    'Period: ' + period + '<extra></extra>',
                        customdata=period_frame['STUDENT_COUNT'],
                        name=period,
                        connectgaps=True
                    )
                ],
                name=period
            )
        )

# Create initial frame with better styling
if len(periods) > 0:
    initial_period_data = gpa_trends[gpa_trends['PERIOD'] == periods[0]]
    
    fig4 = go.Figure(
        data=[
            go.Scatter(
                x=initial_period_data['CITIZENSHIP'],
                y=initial_period_data['AVG_GPA'],
                mode='markers+lines+text',
                marker=dict(
                    size=[20 + (count/5) for count in initial_period_data['STUDENT_COUNT']],
                    color=citizenship_colors[:len(initial_period_data)],
                    line=dict(width=3, color='white'),
                    opacity=0.8,
                    symbol='circle'
                ),
                line=dict(
                    width=4, 
                    color=COLORS['primary'],
                    shape='spline',
                    smoothing=0.3
                ),
                text=[f"{gpa:.2f}" for gpa in initial_period_data['AVG_GPA']],
                textposition='top center',
                textfont=dict(size=14, color=COLORS['text'], family='Arial Black'),
                hovertemplate='<b>%{x}</b><br>' +
                            'Average GPA: %{y:.2f}<br>' +
                            'Students: %{customdata}<br>' +
                            'Period: ' + periods[0] + '<extra></extra>',
                customdata=initial_period_data['STUDENT_COUNT'],
                name=periods[0],
                connectgaps=True
            )
        ],
        frames=frames
    )

    # Enhanced layout with dashboard theme
    fig4.update_layout(
        title=dict(
            text='<b>📈 Academic Performance Evolution Over Time</b><br>' +
                 '<sub>Average GPA trends by citizenship status across semesters</sub><br>' +
                 '<span style="font-size:12px; color:' + COLORS['text_dim'] + ';">💡 Marker size reflects student count | Only periods with 3+ students shown</span>',
            font=dict(size=20, family='Orbitron, monospace', color=COLORS['light']),
            x=0.5,
            y=0.95
        ),
        xaxis=dict(
            title=dict(text='Citizenship Status', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_blue'])),
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            showline=True,
            linewidth=2,
            linecolor=COLORS['text_dim'],
            tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text'])
        ),
        yaxis=dict(
            title=dict(text='Average GPA', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_purple'])),
            range=[1.5, 4.0], 
            gridcolor='rgba(200,200,200,0.2)', 
            gridwidth=1,
            tickmode='linear',
            tick0=1.5,
            dtick=0.25,
            showline=True,
            linewidth=2,
            linecolor=COLORS['text_dim'],
            tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text'])
        ),
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        paper_bgcolor=COLORS['dark_bg'],
        plot_bgcolor='rgba(0,0,0,0)',
        height=700,
        margin=dict(t=150, l=90, r=90, b=80),
        showlegend=False,
        
        # Enhanced animation controls with dashboard theme
        updatemenus=[
            dict(
                type='buttons',
                showactive=False,
                x=0.02,
                xanchor='left',
                y=1.12,
                yanchor='top',
                bgcolor=COLORS['card_bg'],
                bordercolor=COLORS['neon_green'],
                borderwidth=2,
                font=dict(color=COLORS['light']),
                buttons=[
                    dict(
                        label='▶️ Play Animation',
                        method='animate',
                        args=[None, dict(
                            frame=dict(duration=1200, redraw=True), 
                            fromcurrent=True,
                            transition=dict(duration=400, easing='cubic-in-out')
                        )]
                    ),
                    dict(
                        label='⏸️ Pause',
                        method='animate',
                        args=[[None], dict(
                            frame=dict(duration=0, redraw=False), 
                            mode='immediate'
                        )]
                    ),
                    dict(
                        label='🔄 Restart',
                        method='animate',
                        args=[None, dict(
                            frame=dict(duration=1200, redraw=True), 
                            fromcurrent=False,
                            transition=dict(duration=400, easing='cubic-in-out')
                        )]
                    )
                ]
            )
        ],
        
        # Add filter label annotation
        annotations=[
            dict(
                text="🎬 Animation Controls:",
                x=0.02, y=1.18,
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_green'])
            )
        ],
        
        # Enhanced slider with better styling
        sliders=[
            dict(
                active=0,
                yanchor='top',
                xanchor='left',
                currentvalue=dict(
                    font=dict(size=16, family='Arial Black'), 
                    prefix='📅 Current Period: ', 
                    visible=True, 
                    xanchor='center'
                ),
                transition=dict(duration=400, easing='cubic-in-out'),
                pad=dict(b=10, t=60),
                len=0.85,
                x=0.075,
                y=-0.05,
                bgcolor='rgba(255,255,255,0.9)',
                bordercolor=COLORS['neutral'],
                borderwidth=1,
                tickcolor=COLORS['primary'],
                steps=[
                    dict(
                        args=[[period], dict(
                            frame=dict(duration=400, redraw=True), 
                            mode='immediate',
                            transition=dict(duration=400, easing='cubic-in-out')
                        )],
                        label=period,
                        method='animate',
                        value=period
                    ) for period in periods
                ]
            )
        ]
    )

    # Add enhanced reference lines with dashboard colors
    fig4.add_hline(
        y=2.0, 
        line_dash="dot", 
        line_color=COLORS['danger'], 
        line_width=3,
        annotation_text="🎯 Pass Threshold (2.0)",
        annotation_position="top right"
    )

    fig4.add_hline(
        y=3.0, 
        line_dash="dot", 
        line_color=COLORS['success'], 
        line_width=3,
        annotation_text="🌟 Excellence Threshold (3.0)",
        annotation_position="bottom right"
    )

    # Add subtle gradient effect with shapes using dashboard colors
    fig4.add_shape(
        type="rect",
        xref="paper", yref="y",
        x0=0, y0=3.0, x1=1, y1=4.0,
        fillcolor=f"rgba({int(COLORS['success'][1:3], 16)}, {int(COLORS['success'][3:5], 16)}, {int(COLORS['success'][5:7], 16)}, 0.1)",
        layer="below",
        line_width=0,
    )
    
    fig4.add_shape(
        type="rect",
        xref="paper", yref="y",
        x0=0, y0=2.0, x1=1, y1=3.0,
        fillcolor="rgba(52, 152, 219, 0.1)",
        layer="below",
        line_width=0,
    )
    
    fig4.add_shape(
        type="rect",
        xref="paper", yref="y",
        x0=0, y0=1.5, x1=1, y1=2.0,
        fillcolor="rgba(231, 76, 60, 0.1)",
        layer="below",
        line_width=0,
    )

    fig4.show()
    print("✅ Chart 4 completed!")
else:
    print("⚠️ No sufficient data for GPA trends animation")

🎨 Creating Chart 4: Animated GPA Performance Trends...


✅ Chart 4 completed!


# PART 3: ANALYTICS INSIGHTS & DASHBOARD INTEGRATION
## Summary Statistics and Actionable Insights

This section provides comprehensive insights from the data analysis and prepares the foundation for dashboard integration.

In [28]:
# === COMPREHENSIVE ANALYTICS INSIGHTS ===
print("\n" + "="*60)
print("📊 COMPREHENSIVE ANALYTICS INSIGHTS")
print("="*60)

# Analyze and print insights focusing on GPA vs Completion Time
print("\n🎯 GPA vs COMPLETION TIME INSIGHTS:")
print("=" * 50)

print("📊 SUCCESS SCORE DISTRIBUTION:")
success_quartiles = scatter_data['SUCCESS_SCORE'].quantile([0.25, 0.5, 0.75])
print(f"   • Top 25%: Success Score ≥ {success_quartiles[0.75]:.1f}")
print(f"   • Median: {success_quartiles[0.5]:.1f}")
print(f"   • Bottom 25%: Success Score ≤ {success_quartiles[0.25]:.1f}")

print(f"\n🚨 RISK ASSESSMENT BREAKDOWN:")
risk_breakdown = scatter_data['RISK_CATEGORY'].value_counts()
for risk, count in risk_breakdown.items():
    percentage = (count / len(scatter_data)) * 100
    avg_completion = scatter_data[scatter_data['RISK_CATEGORY'] == risk]['COMPLETION_MONTHS'].mean()
    avg_gpa = scatter_data[scatter_data['RISK_CATEGORY'] == risk]['CUMULATIVE_GPA'].mean()
    print(f"   • {risk}: {count} students ({percentage:.1f}%) - Avg: {avg_gpa:.2f} GPA, {avg_completion:.1f} months")

print(f"\n⏰ COMPLETION TIME vs GPA ANALYSIS:")
fast_students = scatter_data[scatter_data['COMPLETION_MONTHS'] <= 30]
slow_students = scatter_data[scatter_data['COMPLETION_MONTHS'] > 48]
print(f"   • Fast completion (≤30 months): {len(fast_students)} students, Avg GPA: {fast_students['CUMULATIVE_GPA'].mean():.2f}")
print(f"   • Slow completion (>48 months): {len(slow_students)} students, Avg GPA: {slow_students['CUMULATIVE_GPA'].mean():.2f}")

print(f"\n🎯 HIGH PERFORMERS vs LOW PERFORMERS:")
high_gpa = scatter_data[(scatter_data['CUMULATIVE_GPA'] > 3.5) & (scatter_data['COMPLETION_MONTHS'] < 5)]
low_gpa = scatter_data[scatter_data['CUMULATIVE_GPA'] < 2.0]
print(f"   • Elite High Achievers (GPA > 3.5 & < 5 months): {len(high_gpa)} students")
print(f"   • Low GPA (<2.0): {len(low_gpa)} students, Avg completion: {low_gpa['COMPLETION_MONTHS'].mean():.1f} months")

print(f"\n🎯 KEY RECOMMENDATIONS:")
high_risk_students = scatter_data[scatter_data['RISK_CATEGORY'] == 'High Risk']
if len(high_risk_students) > 0:
    print(f"   • {len(high_risk_students)} students need immediate academic support")
    
fast_excellent = scatter_data[(scatter_data['COMPLETION_MONTHS'] < 5) & (scatter_data['CUMULATIVE_GPA'] > 3.5)]
if len(fast_excellent) > 0:
    print(f"   • {len(fast_excellent)} students are elite high achievers (GPA > 3.5 + completion < 5 months)")
    
at_risk = scatter_data[(scatter_data['CUMULATIVE_GPA'] < 2.5) & (scatter_data['COMPLETION_MONTHS'] > 42)]
print(f"   • {len(at_risk)} students are struggling (low GPA + slow completion)")

print("✅ Detailed insights analysis completed!")


📊 COMPREHENSIVE ANALYTICS INSIGHTS

🎯 GPA vs COMPLETION TIME INSIGHTS:
📊 SUCCESS SCORE DISTRIBUTION:
   • Top 25%: Success Score ≥ 89.2
   • Median: 84.0
   • Bottom 25%: Success Score ≤ 80.0

🚨 RISK ASSESSMENT BREAKDOWN:
   • Low Risk: 40 students (71.4%) - Avg: 3.25 GPA, 5.9 months
   • Medium Risk: 16 students (28.6%) - Avg: 2.94 GPA, 17.0 months
   • High Risk: 0 students (0.0%) - Avg: nan GPA, nan months

⏰ COMPLETION TIME vs GPA ANALYSIS:
   • Fast completion (≤30 months): 56 students, Avg GPA: 3.16
   • Slow completion (>48 months): 0 students, Avg GPA: nan

🎯 HIGH PERFORMERS vs LOW PERFORMERS:
   • Elite High Achievers (GPA > 3.5 & < 5 months): 5 students
   • Low GPA (<2.0): 0 students, Avg completion: nan months

🎯 KEY RECOMMENDATIONS:
   • 5 students are elite high achievers (GPA > 3.5 + completion < 5 months)
   • 0 students are struggling (low GPA + slow completion)
✅ Detailed insights analysis completed!


In [29]:
# === SUMMARY STATISTICS FOR DASHBOARD INTEGRATION ===
print("\n" + "="*60)
print("📊 KEY METRICS SUMMARY FOR DASHBOARD")
print("="*60)

# Calculate key metrics
total_students = len(df)
avg_completion_time = df['COMPLETION_MONTHS'].mean()
avg_gpa = df['CUMULATIVE_GPA'].mean()
pass_rate = (df['ACADEMIC_OUTCOME'] == 'Pass').mean() * 100
best_citizenship = df.groupby('CITIZENSHIP')['CUMULATIVE_GPA'].mean().idxmax()
best_funding = df.groupby('COURSE FUNDING')['CUMULATIVE_GPA'].mean().idxmax()
fastest_completion = df.groupby('CITIZENSHIP')['COMPLETION_MONTHS'].mean().idxmin()

print(f"📈 Total Students Analyzed: {total_students:,}")
print(f"⏱️ Average Completion Time: {avg_completion_time:.1f} months")
print(f"🎯 Overall Average GPA: {avg_gpa:.2f}")
print(f"✅ Pass Rate (GPA ≥ 2.0): {pass_rate:.1f}%")
print(f"🏆 Top Performing Citizenship: {best_citizenship}")
print(f"💰 Best Funding Type: {best_funding}")
print(f"⚡ Fastest Completion Group: {fastest_completion}")
print(f"📚 Analysis covers enrollment and completion patterns")

print("\n🎨 VISUALIZATION SYSTEM COMPLETED!")
print("Each chart features:")
print("• Chart 1: Population distribution treemap")
print("• Chart 2: Interactive GPA vs completion time correlation analysis")
print("• Chart 3: Course completion time distribution patterns")
print("• Chart 4: Animated GPA trends over semesters")
print("• Interactive elements (dropdowns, sliders, play/pause)")
print("• Comprehensive hover information")
print("• Actionable insights for decision-making")

print(f"\n🔗 DATA PREPARATION STATUS:")
print("• ✅ All datasets loaded and processed")
print("• ✅ Advanced calculated fields generated")
print("• ✅ Risk assessment scoring completed")
print("• ✅ Success prediction metrics calculated")
print("• ✅ Interactive visualizations created")
print("• ✅ Template aesthetic consistency maintained")
print("• ✅ Ready for dashboard integration")

print(f"\n📊 DASHBOARD INTEGRATION READY!")
print("This analysis system can now be integrated with:")
print("• Dash web application framework")
print("• Real-time filtering capabilities")
print("• Student search functionality")
print("• KPI monitoring dashboards")
print("• Administrative decision support tools")


📊 KEY METRICS SUMMARY FOR DASHBOARD
📈 Total Students Analyzed: 285
⏱️ Average Completion Time: 9.5 months
🎯 Overall Average GPA: 3.16
✅ Pass Rate (GPA ≥ 2.0): 95.4%
🏆 Top Performing Citizenship: FOREIGNER
💰 Best Funding Type: Individual - SFC + $1000 SCHOLARSHIP
⚡ Fastest Completion Group: SG PR
📚 Analysis covers enrollment and completion patterns

🎨 VISUALIZATION SYSTEM COMPLETED!
Each chart features:
• Chart 1: Population distribution treemap
• Chart 2: Interactive GPA vs completion time correlation analysis
• Chart 3: Course completion time distribution patterns
• Chart 4: Animated GPA trends over semesters
• Interactive elements (dropdowns, sliders, play/pause)
• Comprehensive hover information
• Actionable insights for decision-making

🔗 DATA PREPARATION STATUS:
• ✅ All datasets loaded and processed
• ✅ Advanced calculated fields generated
• ✅ Risk assessment scoring completed
• ✅ Success prediction metrics calculated
• ✅ Interactive visualizations created
• ✅ Template aesthetic 

# PART 4: INTERACTIVE DASHBOARD IMPLEMENTATION
## Futuristic Analytics Dashboard with Dash Framework

This section integrates all the analysis and charts into a live, interactive web dashboard with search functionality and real-time KPI monitoring.

In [30]:
# === DASHBOARD APP INITIALIZATION (MATCHING TEMPLATE) ===
print("🚀 Initializing Futuristic Analytics Dashboard...")

# Import Dash framework
from dash import Dash, html, dcc, Input, Output, State, callback
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

# ================================
# APP INITIALIZATION WITH DATE PICKER FIX
# ================================
app = Dash(
    __name__,
    external_stylesheets=[
        dbc.themes.CYBORG,  # Futuristic dark theme
        'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css'
    ],
    suppress_callback_exceptions=True
)

app.title = "Student Analytics Dashboard"

# Custom CSS to fix date picker z-index overlay issues
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .DateInput_input, .DateRangePickerInput {
                z-index: 9999 !important;
                position: relative !important;
            }
            .DateRangePicker, .DayPicker {
                z-index: 10000 !important;
                position: absolute !important;
            }
            .SingleDatePicker_picker {
                z-index: 10004 !important;
                position: absolute !important;
            }
            .SingleDatePickerInput {
                z-index: 10006 !important;
                position: relative !important;
            }
            .DayPicker_portal {
                z-index: 10002 !important;
            }
            .DateInput_fang {
                z-index: 10003 !important;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

print("✅ Dashboard framework initialized with date picker overlay fix!")

🚀 Initializing Futuristic Analytics Dashboard...
✅ Dashboard framework initialized with date picker overlay fix!


In [31]:
# === DASHBOARD HEADER COMPONENT (MATCHING TEMPLATE) ===
print("🎨 Creating futuristic header component...")

# Header with enhanced glowing effects - EXACT TEMPLATE MATCH
header = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Div([
                    html.I(className="fas fa-graduation-cap", 
                          style={
                              'fontSize': '1.8rem', 
                              'color': COLORS['neon_blue'],
                              'filter': 'drop-shadow(0 0 8px rgba(0, 245, 255, 0.8))'
                          }),
                ], style={
                    'width': '50px', 'height': '50px', 'borderRadius': '15px',
                    'background': f'linear-gradient(135deg, {COLORS["primary"]} 0%, {COLORS["neon_blue"]} 100%)',
                    'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center',
                    'marginRight': '15px', 
                    'boxShadow': f'0 0 25px {COLORS["glow_shadow"]}, 0 0 50px rgba(139, 95, 191, 0.3)',
                    'border': f'1px solid {COLORS["neon_blue"]}',
                    'animation': 'pulse 2s infinite'
                }),
                html.Div([
                    html.H5("STUDENT ANALYTICS", style={
                        'color': COLORS['light'], 'margin': '0', 'fontWeight': '700',
                        'letterSpacing': '1px', 'fontSize': '0.95rem',
                        'textShadow': f'0 0 10px {COLORS["neon_blue"]}',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    html.P("CONTROL CENTER", style={
                        'color': COLORS['neon_blue'], 'margin': '0', 'fontSize': '0.75rem',
                        'fontWeight': '500', 'letterSpacing': '0.5px',
                        'fontFamily': 'Rajdhani, monospace'
                    })
                ])
            ], style={'display': 'flex', 'alignItems': 'center'})
        ], width=3),
        
        dbc.Col([
            html.H1("ANALYTICS NEXUS", 
                   className="text-center",
                   style={
                       'color': COLORS['light'], 
                       'fontWeight': '900',
                       'fontSize': '2.5rem',
                       'margin': '0',
                       'background': f'linear-gradient(135deg, {COLORS["neon_blue"]} 0%, {COLORS["neon_purple"]} 50%, {COLORS["primary"]} 100%)',
                       'backgroundClip': 'text',
                       'WebkitBackgroundClip': 'text',
                       'WebkitTextFillColor': 'transparent',
                       'letterSpacing': '2px',
                       'textShadow': '0 0 20px rgba(34, 211, 238, 0.6)',
                       'filter': 'drop-shadow(0 0 10px rgba(34, 211, 238, 0.8))',
                       'fontFamily': 'Orbitron, monospace'
                   })
        ], width=6),
        
        dbc.Col([
            html.Div([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-circle", style={
                            'color': COLORS['neon_green'], 'fontSize': '0.6rem',
                            'animation': 'blink 1.5s infinite'
                        }),
                        html.Span("LIVE", style={
                            'color': COLORS['neon_green'], 'fontSize': '0.8rem', 
                            'marginLeft': '6px', 'fontWeight': '700',
                            'textShadow': f'0 0 8px {COLORS["neon_green"]}',
                            'fontFamily': 'Orbitron, monospace'
                        })
                    ], style={'display': 'flex', 'alignItems': 'center', 'marginBottom': '4px'}),
                    html.Span(id="live-time", style={
                        'color': COLORS['text'], 'fontSize': '0.85rem',
                        'fontFamily': 'Rajdhani, monospace', 'fontWeight': '500'
                    })
                ], style={'textAlign': 'right'})
            ])
        ], width=3)
    ], className="align-items-center"),
], fluid=True, className="py-4", 
   style={
       'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)', 
       'marginBottom': '30px',
       'borderRadius': '0 0 30px 30px',
       'boxShadow': f'0 0 40px {COLORS["glow_shadow"]}, 0 0 50px rgba(26, 26, 46, 0.8)',
       'border': f'1px solid {COLORS["neon_blue"]}20',
       'position': 'relative',
       'overflow': 'hidden'
   })

print("✅ Header component created!")

🎨 Creating futuristic header component...
✅ Header component created!


In [32]:
# === MAIN LAYOUT (EXACT TEMPLATE MATCH) ===
print("🎨 Creating main layout to match template...")

# Chart containers with enhanced glowing design (matching template)
chart_style = {
    'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)',
    'border': f'2px solid {COLORS["neon_blue"]}40',
    'borderRadius': '25px',
    'padding': '30px',
    'marginBottom': '30px',
    'boxShadow': f'0 0 40px {COLORS["glow_shadow"]}, 0 15px 60px rgba(26, 26, 46, 0.8)',
    'backdropFilter': 'blur(15px)',
    'position': 'relative',
    'zIndex': '1',
    'overflow': 'hidden',
    'transition': 'all 0.3s ease'
}

# ================================
# MAIN LAYOUT WITH LEFT-RIGHT ORIENTATION (EXACT TEMPLATE MATCH)
# ================================
app.layout = html.Div([
    # Global CSS for futuristic fonts and date picker fix
    html.Link(
        rel='stylesheet',
        href='https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&family=Rajdhani:wght@300;400;500;600;700&family=Space+Mono:wght@400;700&display=swap'
    ),
    
    # Live update interval
    dcc.Interval(id='interval-update', interval=60000, n_intervals=0),
    
    # Data store
    dcc.Store(id='filtered-data'),
    
    # Header (full width)
    header,
    
    # Main Content Container with Left-Right Layout
    dbc.Container([
        dbc.Row([
            # LEFT SIDE - Search, Student Data, and KPIs (4/12 columns)
            dbc.Col([
                # Search Section
                html.Div([
                    html.Label("🔍 SEARCH STUDENT ID", style={
                        'color': COLORS['pink'], 'fontWeight': '700', 'marginBottom': '15px',
                        'fontSize': '1rem', 'letterSpacing': '1px',
                        'textShadow': f'0 0 8px {COLORS["pink"]}',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    dbc.Input(
                        id='search-box',
                        placeholder="Enter Student ID (e.g., STU00001)",
                        type="text",
                        style={
                            'backgroundColor': COLORS['sidebar_bg'], 'color': COLORS['light'], 
                            'border': f'2px solid {COLORS["pink"]}60', 'borderRadius': '10px',
                            'boxShadow': f'inset 0 0 10px rgba(236, 72, 153, 0.2)',
                            'fontFamily': 'Rajdhani, monospace',
                            'fontSize': '1rem'
                        }
                    )
                ], style={
                    'backgroundColor': COLORS['card_bg'], 'padding': '20px', 'borderRadius': '20px',
                    'boxShadow': f'0 0 30px rgba(236, 72, 153, 0.4), 0 8px 32px rgba(26, 26, 46, 0.6)', 
                    'border': f'2px solid {COLORS["pink"]}40',
                    'backdropFilter': 'blur(10px)',
                    'marginBottom': '20px'
                }),
                
                # Student Data Section
                html.Div([
                    html.H4("📊 STUDENT DATA", 
                           style={
                               'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                               'textTransform': 'uppercase'
                           }),
                    html.Div(id='student-data-table')
                ], style={**chart_style, 'height': '300px', 'overflow': 'auto', 'marginBottom': '20px'}),
                
                # KPI Cards Section (Compact vertical layout)
                html.Div([
                    html.H4("📈 KEY METRICS", 
                           style={
                               'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                               'textTransform': 'uppercase'
                           }),
                    html.Div(id='kpi-cards-left')
                ], style=chart_style)
                
            ], width=4),  # Left side takes 4/12 columns
            
            # RIGHT SIDE - Scrollable Charts (8/12 columns)
            dbc.Col([
                # Simple Date Filter Row with Fixed Z-Index
                html.Div([
                    html.Span("📅 Filter: ", style={
                        'color': COLORS['secondary'], 'fontWeight': '700', 'marginRight': '10px',
                        'fontFamily': 'Orbitron, monospace', 'fontSize': '0.9rem'
                    }),
                    html.Div([
                        dcc.DatePickerSingle(
                            id='start-date-picker',
                            date=datetime(2022, 4, 1),  # Start of actual data range
                            display_format='YYYY-MM-DD',
                            style={
                                'display': 'inline-block', 
                                'marginRight': '10px',
                                'zIndex': '9999'  # High z-index for calendar dropdown
                            }
                        )
                    ], style={
                        'display': 'inline-block', 
                        'position': 'relative',
                        'zIndex': '9999'  # Container z-index
                    }),
                    html.Span(" to ", style={'color': COLORS['light'], 'margin': '0 5px'}),
                    html.Div([
                        dcc.DatePickerSingle(
                            id='end-date-picker',
                            date=datetime(2025, 4, 24),  # End of actual data range
                            display_format='YYYY-MM-DD',
                            style={
                                'display': 'inline-block', 
                                'marginRight': '15px',
                                'zIndex': '9999'  # High z-index for calendar dropdown
                            }
                        )
                    ], style={
                        'display': 'inline-block', 
                        'position': 'relative',
                        'zIndex': '9999'  # Container z-index
                    }),
                    dbc.Button("All Time", id="btn-all-years", color="outline-info", size="sm",
                              style={'fontFamily': 'Rajdhani, monospace', 'fontSize': '0.8rem'})
                ], style={
                    'backgroundColor': COLORS['card_bg'], 
                    'padding': '10px 15px', 
                    'borderRadius': '10px',
                    'border': f'1px solid {COLORS["secondary"]}40', 
                    'marginBottom': '15px',
                    'display': 'flex', 
                    'alignItems': 'center', 
                    'flexWrap': 'wrap',
                    'position': 'relative',
                    'zIndex': '1000'  # Base z-index for filter container
                }),
                
                # Scrollable Charts Container
                html.Div([
                    # Chart 1 - Your Treemap
                    html.Div([
                        html.H4("POPULATION DISTRIBUTION", 
                               style={
                                   'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='chart-1', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 2 - Your Scatter
                    html.Div([
                        html.H4("GPA vs COMPLETION ANALYSIS", 
                               style={
                                   'color': COLORS['neon_purple'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_purple"]}, 0 0 30px {COLORS["neon_purple"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='chart-2', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 3 - Your Boxplot
                    html.Div([
                        html.H4("COMPLETION TIME ANALYSIS", 
                               style={
                                   'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='chart-3', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 4 - Your Animation
                    html.Div([
                        html.H4("TEMPORAL TRENDS", 
                               style={
                                   'color': COLORS['accent'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["accent"]}, 0 0 30px {COLORS["accent"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='chart-4', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style=chart_style)
                    
                ], style={
                    'height': '90vh',  # Set viewport height
                    'overflowY': 'auto',  # Enable vertical scrolling
                    'paddingRight': '15px',  # Space for scrollbar
                    'scrollbarWidth': 'thin',  # Thin scrollbar for Firefox
                    'scrollbarColor': f'{COLORS["neon_blue"]} {COLORS["sidebar_bg"]}'  # Custom scrollbar colors
                })
                
            ], width=8)  # Right side takes 8/12 columns
            
        ], className="g-4")
    ], fluid=True)
], style={
    'backgroundColor': COLORS['dark_bg'], 
    'minHeight': '100vh', 
    'padding': '0',
    'fontFamily': 'Rajdhani, Orbitron, monospace'
})

print("✅ Main layout created to match template exactly!")

🎨 Creating main layout to match template...
✅ Main layout created to match template exactly!


In [33]:
# === DASHBOARD CALLBACKS (MATCHING TEMPLATE PATTERN) ===
print("🔗 Creating callbacks to match template with your actual data...")

# Main callback to update student data, KPIs, and filtered data
@app.callback(
    [Output('student-data-table', 'children'),
     Output('kpi-cards-left', 'children'),
     Output('filtered-data', 'data')],
    [Input('search-box', 'value'),
     Input('start-date-picker', 'date'),
     Input('end-date-picker', 'date'),
     Input('btn-all-years', 'n_clicks')]
)
def update_dashboard(search_value, start_date, end_date, btn_clicks):
    from dash import callback_context
    
    # Start with full dataset (your actual data)
    filtered_df = df.copy()
    
    # Handle "All Time" button
    if callback_context.triggered and 'btn-all-years' in callback_context.triggered[0]['prop_id']:
        start_date = None
        end_date = None
    
    # Apply ONLY date filters for charts and KPIs (NOT search filter)
    chart_filtered_df = filtered_df.copy()
    if start_date and 'COMMENCEMENT DATE' in chart_filtered_df.columns:
        chart_filtered_df = chart_filtered_df[pd.to_datetime(chart_filtered_df['COMMENCEMENT DATE']) >= pd.to_datetime(start_date)]
    if end_date and 'COMMENCEMENT DATE' in chart_filtered_df.columns:
        chart_filtered_df = chart_filtered_df[pd.to_datetime(chart_filtered_df['COMMENCEMENT DATE']) <= pd.to_datetime(end_date)]
    
    # Calculate KPIs from date-filtered data only
    total_students = len(chart_filtered_df)
    avg_gpa = chart_filtered_df['CUMULATIVE_GPA'].mean() if len(chart_filtered_df) > 0 and 'CUMULATIVE_GPA' in chart_filtered_df.columns else 0
    avg_completion = chart_filtered_df['COMPLETION_MONTHS'].mean() if len(chart_filtered_df) > 0 and 'COMPLETION_MONTHS' in chart_filtered_df.columns else 0
    
    # Pass rate calculation (assuming GPA >= 2.0 is pass)
    if len(chart_filtered_df) > 0 and 'CUMULATIVE_GPA' in chart_filtered_df.columns:
        pass_rate = (chart_filtered_df['CUMULATIVE_GPA'] >= 2.0).mean() * 100
    else:
        pass_rate = 0
    
    # Create KPI display (template style with your metrics)
    kpi_display = html.Div([
        html.Div([
            html.H3(f"{total_students:,}", style={
                'color': COLORS['neon_blue'], 'fontWeight': '900', 'margin': '0',
                'fontFamily': 'Orbitron, monospace', 'fontSize': '2rem'
            }),
            html.P("Total Students", style={
                'color': COLORS['text'], 'margin': '5px 0 15px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            })
        ]),
        html.Div([
            html.H3(f"{avg_gpa:.2f}" if avg_gpa > 0 else "N/A", style={
                'color': COLORS['neon_green'], 'fontWeight': '900', 'margin': '0',
                'fontFamily': 'Orbitron, monospace', 'fontSize': '2rem'
            }),
            html.P("Average GPA", style={
                'color': COLORS['text'], 'margin': '5px 0 15px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            })
        ]),
        html.Div([
            html.H3(f"{avg_completion:.1f}" if avg_completion > 0 else "N/A", style={
                'color': COLORS['accent'], 'fontWeight': '900', 'margin': '0',
                'fontFamily': 'Orbitron, monospace', 'fontSize': '2rem'
            }),
            html.P("Avg Completion (months)", style={
                'color': COLORS['text'], 'margin': '5px 0 15px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            })
        ]),
        html.Div([
            html.H3(f"{pass_rate:.1f}%" if pass_rate > 0 else "N/A", style={
                'color': COLORS['neon_purple'], 'fontWeight': '900', 'margin': '0',
                'fontFamily': 'Orbitron, monospace', 'fontSize': '2rem'
            }),
            html.P("Pass Rate", style={
                'color': COLORS['text'], 'margin': '5px 0 15px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            })
        ])
    ])
    
    # Create student data table - show first 10 by default, filter by search if provided
    if search_value:
        # Apply search filter only for display
        search_filtered = df[df['STUDENT ID'].astype(str).str.contains(str(search_value), na=False, case=False)]
        display_students = search_filtered.head(10)  # Show top 10 matches
    else:
        # Show first 10 students by default
        display_students = df.head(10)
    
    student_cards = []
    for _, row in display_students.iterrows():
        # Build card with available columns
        card_content = [
            html.P(f"ID: {row['STUDENT ID']}", style={
                'color': COLORS['neon_blue'], 'fontWeight': 'bold', 'margin': '5px 0',
                'fontFamily': 'Orbitron, monospace'
            })
        ]
        
        # Add available information
        if 'GENDER' in row and pd.notna(row['GENDER']):
            card_content.append(html.P(f"Gender: {row['GENDER']}", style={
                'color': COLORS['text'], 'margin': '3px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        if 'CITIZENSHIP' in row and pd.notna(row['CITIZENSHIP']):
            card_content.append(html.P(f"Citizenship: {row['CITIZENSHIP']}", style={
                'color': COLORS['text'], 'margin': '3px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        if 'CUMULATIVE_GPA' in row and pd.notna(row['CUMULATIVE_GPA']):
            card_content.append(html.P(f"GPA: {row['CUMULATIVE_GPA']:.2f}", style={
                'color': COLORS['neon_green'], 'fontWeight': 'bold', 'margin': '3px 0',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        if 'COURSE FUNDING' in row and pd.notna(row['COURSE FUNDING']):
            card_content.append(html.P(f"Funding: {row['COURSE FUNDING']}", style={
                'color': COLORS['text'], 'margin': '3px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        if 'PERFORMANCE_TIER' in row and pd.notna(row['PERFORMANCE_TIER']):
            card_content.append(html.P(f"Performance: {row['PERFORMANCE_TIER']}", style={
                'color': COLORS['accent'], 'fontWeight': 'bold', 'margin': '3px 0',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        # Add separator
        card_content.append(html.Hr(style={'borderColor': COLORS['text_dim'], 'margin': '10px 0'}))
        
        card = html.Div(card_content, style={
            'padding': '10px', 'margin': '5px 0',
            'border': f'1px solid {COLORS["text_dim"]}40',
            'borderRadius': '10px',
            'backgroundColor': f'{COLORS["card_bg"]}60'
        })
        student_cards.append(card)
    
    student_table = html.Div(student_cards)
    
    # Return data for charts (date-filtered only, no search filter)
    return student_table, kpi_display, chart_filtered_df.to_dict('records')

# Chart callbacks to display your existing figures with dynamic data AND preserve your interactive filters
@app.callback(Output('chart-1', 'figure'), Input('filtered-data', 'data'))
def update_chart_1(filtered_data):
    if not filtered_data:
        return fig1  # Return original with all your filters intact
    
    # Convert filtered data back to DataFrame
    filtered_df = pd.DataFrame(filtered_data)
    
    if len(filtered_df) == 0:
        # Create EMPTY chart with "No Data" message
        fig_empty = px.treemap(
            pd.DataFrame({'x': ['No Data Found'], 'y': [1]}),
            path=['x'],
            values='y',
            title='<b>🚫 No Students Found in Selected Date Range</b><br><sub>Adjust your date filters or click "All Time" to see all data</sub>'
        )
        fig_empty.update_layout(
            font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
            title_font_size=18,
            title_x=0.5,
            title_font_color=COLORS['danger'],
            paper_bgcolor=COLORS['dark_bg'],
            height=700,
            margin=dict(t=150, l=20, r=20, b=20)
        )
        fig_empty.update_traces(
            textfont_color=COLORS['danger'],
            marker_colorscale=[[0, COLORS['danger']], [1, COLORS['danger']]]
        )
        return fig_empty
    
    # Regenerate treemap with filtered data but KEEP the original structure
    treemap_data = filtered_df.groupby(['CITIZENSHIP', 'GENDER', 'COURSE FUNDING']).size().reset_index(name='COUNT')
    treemap_data['PERCENTAGE'] = (treemap_data['COUNT'] / len(filtered_df) * 100).round(1)
    
    fig_updated = px.treemap(
        treemap_data,
        path=['CITIZENSHIP', 'GENDER', 'COURSE FUNDING'],
        values='COUNT',
        title=f'<b>🌍 Student Population Breakdown ({len(filtered_df)} students)</b><br><sub>Distribution by Citizenship, Gender, and Funding Type • Click to drill down</sub>',
        color='COUNT',
        color_continuous_scale='Viridis',
        hover_data={'PERCENTAGE': ':.1f%'}
    )
    
    fig_updated.update_layout(
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        title_font_size=18,
        title_x=0.5,
        title_font_color=COLORS['light'],
        paper_bgcolor=COLORS['dark_bg'],
        height=700,
        margin=dict(t=150, l=20, r=20, b=20)
    )
    
    fig_updated.update_traces(
        textinfo='label+value+percent parent',
        textfont_size=11,
        textfont_color=COLORS['light'],
        hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{customdata[0]:.1f}%<extra></extra>'
    )
    
    return fig_updated

@app.callback(Output('chart-2', 'figure'), Input('filtered-data', 'data'))
def update_chart_2(filtered_data):
    if not filtered_data:
        return fig2  # Return original with ALL your interactive features
    
    # Convert filtered data back to DataFrame
    filtered_df = pd.DataFrame(filtered_data)
    
    if len(filtered_df) == 0:
        # Create EMPTY scatter plot with "No Data" message
        fig_empty = go.Figure()
        fig_empty.add_trace(go.Scatter(
            x=[0], y=[2.5], mode='text',
            text=['No Data Found<br>Adjust date filters or click "All Time"'],
            textfont=dict(size=20, color=COLORS['danger'], family='Orbitron, monospace'),
            showlegend=False
        ))
        fig_empty.update_layout(
            title=dict(
                text='<b>🚫 No Students Found in Selected Date Range</b><br><sub>GPA vs Completion Time Analysis - No data to display</sub>',
                font=dict(size=20, family='Orbitron, monospace', color=COLORS['danger']),
                x=0.5, y=0.95
            ),
            xaxis=dict(
                title=dict(text='Completion Time (Months)', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_blue'])),
                tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
                gridcolor='rgba(200,200,200,0.2)', gridwidth=1, range=[0, 10]
            ),
            yaxis=dict(
                title=dict(text='Cumulative GPA', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_purple'])),
                tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
                gridcolor='rgba(200,200,200,0.2)', gridwidth=1, range=[1.0, 4.0]
            ),
            font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
            paper_bgcolor=COLORS['dark_bg'], plot_bgcolor='rgba(0,0,0,0)',
            height=700, margin=dict(t=150, l=80, r=120, b=80), showlegend=False
        )
        return fig_empty
    
    # Recreate your FULL interactive scatter plot with ALL original features
    scatter_data = filtered_df.dropna(subset=['CUMULATIVE_GPA', 'COMPLETION_MONTHS'])
    
    if len(scatter_data) == 0:
        return fig2
    
    # Fix None values in SUCCESS_SCORE and RISK_SCORE
    scatter_data = scatter_data.copy()
    if 'SUCCESS_SCORE' not in scatter_data.columns or scatter_data['SUCCESS_SCORE'].isna().all():
        # Create success score if missing or all None
        scatter_data['SUCCESS_SCORE'] = (scatter_data['CUMULATIVE_GPA'] / 4.0 * 50) + \
                                       ((scatter_data['COMPLETION_MONTHS'].max() - scatter_data['COMPLETION_MONTHS']) / 
                                        scatter_data['COMPLETION_MONTHS'].max() * 50)
    else:
        # Fill None values with calculated score
        mask = scatter_data['SUCCESS_SCORE'].isna()
        scatter_data.loc[mask, 'SUCCESS_SCORE'] = (scatter_data.loc[mask, 'CUMULATIVE_GPA'] / 4.0 * 50) + \
                                                 ((scatter_data['COMPLETION_MONTHS'].max() - scatter_data.loc[mask, 'COMPLETION_MONTHS']) / 
                                                  scatter_data['COMPLETION_MONTHS'].max() * 50)
    
    if 'RISK_SCORE' not in scatter_data.columns or scatter_data['RISK_SCORE'].isna().all():
        scatter_data['RISK_SCORE'] = 1 - (scatter_data['SUCCESS_SCORE'] / 100)
    else:
        # Fill None values
        mask = scatter_data['RISK_SCORE'].isna()
        scatter_data.loc[mask, 'RISK_SCORE'] = 1 - (scatter_data.loc[mask, 'SUCCESS_SCORE'] / 100)
        
    if 'RISK_CATEGORY' not in scatter_data.columns:
        scatter_data['RISK_CATEGORY'] = pd.cut(scatter_data['RISK_SCORE'], 
                                             bins=[0, 0.3, 0.7, 1.0], 
                                             labels=['Low Risk', 'Medium Risk', 'High Risk'])
    
    fig_updated = go.Figure()
    
    # Recreate ALL your original traces with citizenship filtering
    citizenship_types = scatter_data['CITIZENSHIP'].unique()
    for i, citizenship in enumerate(citizenship_types):
        citizenship_subset = scatter_data[scatter_data['CITIZENSHIP'] == citizenship]
        
        # Regular traces
        fig_updated.add_trace(go.Scatter(
            x=citizenship_subset['COMPLETION_MONTHS'],
            y=citizenship_subset['CUMULATIVE_GPA'],
            mode='markers',
            marker=dict(
                size=12 + (citizenship_subset['RISK_SCORE'] * 4),
                color=citizenship_subset['SUCCESS_SCORE'],  # Now guaranteed to be numeric
                colorscale='Viridis',
                showscale=True if i == 0 else False,
                colorbar=dict(
                    title="Success<br>Score<br>(0-100)",
                    titlefont=dict(size=12, family='Arial Black'),
                    tickfont=dict(size=10),
                    len=0.8
                ) if i == 0 else None,
                line=dict(width=2, color='white'),
                opacity=0.8,
                symbol='circle'
            ),
            text=[f"Student ID: {row['STUDENT ID']}<br>" +
                  f"GPA: {row['CUMULATIVE_GPA']:.2f}<br>" +
                  f"Completion: {row['COMPLETION_MONTHS']:.1f} months<br>" +
                  f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
                  f"Risk Category: {row['RISK_CATEGORY']}<br>" +
                  f"Citizenship: {row['CITIZENSHIP']}<br>" +
                  f"Gender: {row['GENDER']}<br>" +
                  f"Funding: {row['COURSE FUNDING']}"
                  for _, row in citizenship_subset.iterrows()],
            hovertemplate='%{text}<extra></extra>',
            name=f'{citizenship} ({len(citizenship_subset)} students)',
            visible=True
        ))
    
    # Add HIGH ACHIEVERS traces (preserving your logic)
    for i, citizenship in enumerate(citizenship_types):
        high_achievers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                     (scatter_data['CUMULATIVE_GPA'] > 3.5) &
                                     (scatter_data['COMPLETION_MONTHS'] < 5)]
        
        fig_updated.add_trace(go.Scatter(
            x=high_achievers['COMPLETION_MONTHS'],
            y=high_achievers['CUMULATIVE_GPA'],
            mode='markers',
            marker=dict(
                size=12 + (high_achievers['RISK_SCORE'] * 4) if 'RISK_SCORE' in high_achievers.columns else 12,
                color=high_achievers['SUCCESS_SCORE'] if 'SUCCESS_SCORE' in high_achievers.columns else citizenship_colors[i % len(citizenship_colors)],
                colorscale='Viridis',
                showscale=False,
                line=dict(width=2, color='white'),
                opacity=0.8,
                symbol='circle'
            ),
            text=[f"Student ID: {row['STUDENT ID']}<br>" +
                  f"GPA: {row['CUMULATIVE_GPA']:.2f} ⭐⭐<br>" +
                  f"Completion: {row['COMPLETION_MONTHS']:.1f} months 🚀<br>" +
                  f"Success Score: {row.get('SUCCESS_SCORE', 'N/A')}<br>" +
                  f"Risk Category: {row.get('RISK_CATEGORY', 'N/A')}<br>" +
                  f"Citizenship: {row['CITIZENSHIP']}<br>" +
                  f"Gender: {row['GENDER']}<br>" +
                  f"Funding: {row['COURSE FUNDING']}"
                  for _, row in high_achievers.iterrows()],
            hovertemplate='%{text}<extra></extra>',
            name=f'{citizenship} High Achievers ({len(high_achievers)} students)',
            visible=False
        ))
    
    # Add UNDERPERFORMERS traces (preserving your logic)
    for i, citizenship in enumerate(citizenship_types):
        underperformers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                      (scatter_data['CUMULATIVE_GPA'] < 2.5)]
        
        fig_updated.add_trace(go.Scatter(
            x=underperformers['COMPLETION_MONTHS'],
            y=underperformers['CUMULATIVE_GPA'],
            mode='markers',
            marker=dict(
                size=12 + (underperformers['RISK_SCORE'] * 4) if 'RISK_SCORE' in underperformers.columns else 12,
                color=underperformers['SUCCESS_SCORE'] if 'SUCCESS_SCORE' in underperformers.columns else citizenship_colors[i % len(citizenship_colors)],
                colorscale='Viridis',
                showscale=False,
                line=dict(width=2, color='white'),
                opacity=0.8,
                symbol='circle'
            ),
            text=[f"Student ID: {row['STUDENT ID']}<br>" +
                  f"GPA: {row['CUMULATIVE_GPA']:.2f} 🚨<br>" +
                  f"Completion: {row['COMPLETION_MONTHS']:.1f} months<br>" +
                  f"Success Score: {row.get('SUCCESS_SCORE', 'N/A')}<br>" +
                  f"Risk Category: {row.get('RISK_CATEGORY', 'N/A')}<br>" +
                  f"Citizenship: {row['CITIZENSHIP']}<br>" +
                  f"Gender: {row['GENDER']}<br>" +
                  f"Funding: {row['COURSE FUNDING']}"
                  for _, row in underperformers.iterrows()],
            hovertemplate='%{text}<extra></extra>',
            name=f'{citizenship} Underperformers ({len(underperformers)} students)',
            visible=False
        ))
    
    # Recreate ALL your dropdown buttons (preserving your original logic)
    num_citizenship = len(citizenship_types)
    dropdown_buttons = [
        # Show all citizenship types (regular view)
        dict(label='🌍 All Citizenship Types', 
             method='update', 
             args=[{'visible': [True] * num_citizenship + [False] * (num_citizenship * 2)}]),
    ]
    
    # Add individual citizenship options (regular view)
    for i, citizenship in enumerate(citizenship_types):
        visibility = [False] * (num_citizenship * 3)
        visibility[i] = True
        dropdown_buttons.append(
            dict(label=f'🎯 {citizenship} Only',
                 method='update',
                 args=[{'visible': visibility}])
        )
    
    # Add high achievers view
    high_achiever_visibility = [False] * num_citizenship + [True] * num_citizenship + [False] * num_citizenship
    dropdown_buttons.append(
        dict(label='⭐ High Achievers (GPA > 3.5 & < 5 months)', 
             method='update', 
             args=[{'visible': high_achiever_visibility}])
    )
    
    # Add underperformers view
    underperformer_visibility = [False] * (num_citizenship * 2) + [True] * num_citizenship
    dropdown_buttons.append(
        dict(label='🚨 Underperformers (GPA < 2.5)', 
             method='update', 
             args=[{'visible': underperformer_visibility}])
    )
    
    fig_updated.update_layout(
        title=dict(
            text=f'<b>🎯 Interactive GPA vs Completion Time Analysis ({len(filtered_df)} students)</b><br>' +
                 '<sub>Success score by color • Risk level by marker size • Use filters to explore different groups</sub>',
            font=dict(size=20, family='Orbitron, monospace', color=COLORS['light']),
            x=0.5,
            y=0.95
        ),
        xaxis=dict(
            title=dict(text='Completion Time (Months)', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_blue'])),
            tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            range=[0, max(scatter_data['COMPLETION_MONTHS']) + 5] if len(scatter_data) > 0 else [0, 10]
        ),
        yaxis=dict(
            title=dict(text='Cumulative GPA', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_purple'])),
            tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            range=[1.0, 4.0]
        ),
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        paper_bgcolor=COLORS['dark_bg'],
        plot_bgcolor='rgba(0,0,0,0)',
        height=700,
        margin=dict(t=150, l=80, r=120, b=80),
        showlegend=True,
        legend=dict(
            orientation='v',
            yanchor='top',
            y=0.98,
            xanchor='left',
            x=1.02,
            bgcolor=COLORS['card_bg'],
            bordercolor=COLORS['neon_blue'],
            borderwidth=1,
            font=dict(color=COLORS['light'])
        ),
        
        # PRESERVE your interactive dropdown menu
        updatemenus=[
            dict(
                buttons=dropdown_buttons,
                direction="down",
                showactive=True,
                x=0.02,
                xanchor="left",
                y=1.12,
                yanchor="top",
                bgcolor=COLORS['card_bg'],
                bordercolor=COLORS['neon_green'],
                borderwidth=2,
                font=dict(color=COLORS['light'])
            )
        ],
        
        # PRESERVE your filter label
        annotations=[
            dict(
                text="🔍 Filter by Group:",
                x=0.02, y=1.18,
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_green'])
            )
        ]
    )
    
    # Add reference lines (preserving your original styling)
    fig_updated.add_hline(y=2.0, line_dash="dot", line_color=COLORS['danger'], line_width=2, opacity=0.8, 
                          annotation_text="Pass Threshold (2.0)")
    fig_updated.add_hline(y=3.5, line_dash="dot", line_color=COLORS['success'], line_width=2, opacity=0.8, 
                          annotation_text="Excellence Threshold (3.5)")
    
    return fig_updated

@app.callback(Output('chart-3', 'figure'), Input('filtered-data', 'data'))
def update_chart_3(filtered_data):
    if not filtered_data:
        return fig3  # Return original with ALL your dropdowns
    
    # Convert filtered data back to DataFrame
    filtered_df = pd.DataFrame(filtered_data)
    
    if len(filtered_df) == 0:
        # Create EMPTY box plot with "No Data" message
        fig_empty = go.Figure()
        fig_empty.add_trace(go.Scatter(
            x=[0], y=[15], mode='text',
            text=['No Data Found<br>Adjust date filters or click "All Time"'],
            textfont=dict(size=20, color=COLORS['danger'], family='Orbitron, monospace'),
            showlegend=False
        ))
        fig_empty.update_layout(
            title='<b>🚫 No Students Found in Selected Date Range</b><br><sub>Completion Time Distribution - No data to display</sub>',
            title_font_size=18, title_x=0.5, title_font_color=COLORS['danger'],
            yaxis_title='Completion Time (Months)', xaxis_title='Citizenship Status',
            font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
            paper_bgcolor=COLORS['dark_bg'], plot_bgcolor='rgba(0,0,0,0)',
            height=700, margin=dict(t=150, l=80, r=80, b=80), showlegend=False,
            yaxis=dict(gridcolor='rgba(200,200,200,0.2)', gridwidth=1, range=[0, 30],
                      title_font_color=COLORS['neon_green'], tickfont=dict(color=COLORS['text'])),
            xaxis=dict(gridcolor='rgba(200,200,200,0.2)', gridwidth=1,
                      title_font_color=COLORS['neon_blue'], tickfont=dict(color=COLORS['text']))
        )
        return fig_empty
    
    # Regenerate box plot with filtered data but PRESERVE all your dropdown functionality
    completion_data = filtered_df.dropna(subset=['COMPLETION_MONTHS'])
    completion_data = completion_data[completion_data['COMPLETION_MONTHS'] > 0]
    completion_data = completion_data[completion_data['COMPLETION_MONTHS'] <= 60]
    
    if len(completion_data) == 0:
        return fig3
    
    fig_updated = go.Figure()
    
    # Add box plot for each citizenship type in filtered data (preserving your structure)
    for i, citizenship in enumerate(completion_data['CITIZENSHIP'].unique()):
        citizenship_data = completion_data[completion_data['CITIZENSHIP'] == citizenship]
        
        fig_updated.add_trace(
            go.Box(
                y=citizenship_data['COMPLETION_MONTHS'],
                name=citizenship,
                marker_color=citizenship_colors[i % len(citizenship_colors)],
                boxpoints='outliers',
                jitter=0.3,
                pointpos=-1.8,
                marker=dict(
                    size=4,
                    opacity=0.6,
                    line=dict(width=1, color='white')
                ),
                hovertemplate='<b>%{fullData.name}</b><br>' +
                             'Completion Time: %{y:.1f} months<br>' +
                             'Student Count: ' + str(len(citizenship_data)) + '<br>' +
                             '<extra></extra>',
                visible=True
            )
        )
    
    # Recreate YOUR dropdown buttons (preserving original logic)
    all_visible = [True] * len(completion_data['CITIZENSHIP'].unique())
    dropdown_buttons = [
        dict(label='All Citizenship Types',
             method='update',
             args=[{'visible': all_visible}])
    ]
    
    # Add individual citizenship type options (preserving your logic)
    for i, citizenship in enumerate(completion_data['CITIZENSHIP'].unique()):
        visibility = [False] * len(completion_data['CITIZENSHIP'].unique())
        visibility[i] = True
        dropdown_buttons.append(
            dict(label=f'{citizenship} Only',
                 method='update',
                 args=[{'visible': visibility}])
        )
    
    fig_updated.update_layout(
        title=f'<b>🔍 Course Completion Time Distribution ({len(filtered_df)} students)</b><br><sub>Interactive box plot analysis of completion duration by citizenship status</sub>',
        title_font_size=18,
        title_x=0.5,
        title_font_color=COLORS['light'],
        yaxis_title='Completion Time (Months)',
        xaxis_title='Citizenship Status',
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        paper_bgcolor=COLORS['dark_bg'],
        plot_bgcolor='rgba(0,0,0,0)',
        height=700,
        margin=dict(t=150, l=80, r=80, b=80),
        showlegend=False,
        
        # PRESERVE your dropdown functionality
        updatemenus=[
            dict(
                buttons=dropdown_buttons,
                direction='down',
                showactive=True,
                x=0.02,
                xanchor='left',
                y=1.12,
                yanchor='top',
                bgcolor=COLORS['card_bg'],
                bordercolor=COLORS['neon_purple'],
                borderwidth=2,
                font=dict(color=COLORS['light'])
            )
        ],
        annotations=[
            dict(
                text="🔍 Filter Options:",
                x=0.02, y=1.18,
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_purple'])
            )
        ],
        yaxis=dict(
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            range=[0, 60],
            title_font_color=COLORS['neon_green'],
            tickfont=dict(color=COLORS['text'])
        ),
        xaxis=dict(
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            title_font_color=COLORS['neon_blue'],
            tickfont=dict(color=COLORS['text'])
        )
    )
    
    return fig_updated

@app.callback(Output('chart-4', 'figure'), Input('filtered-data', 'data'))
def update_chart_4(filtered_data):
    # For the animated chart, preserve it exactly as you made it
    if not filtered_data:
        return fig4
    
    filtered_df = pd.DataFrame(filtered_data)
    total_filtered = len(filtered_df)
    total_original = len(df)
    
    # Clone the original figure and update title to show filter status
    fig_updated = go.Figure(fig4)
    fig_updated.update_layout(
        title=f'<b>📈 Temporal Trends ({total_filtered}/{total_original} students)</b><br><sub>Animated progression over academic periods</sub>'
    )
    
    return fig_updated

# Live time update callback
@app.callback(
    Output('live-time', 'children'),
    Input('interval-update', 'n_intervals')
)
def update_time(n):
    from datetime import datetime
    return datetime.now().strftime('%H:%M:%S')

# "All Time" button callback to reset date pickers to original data range
@app.callback(
    [Output('start-date-picker', 'date'),
     Output('end-date-picker', 'date')],
    Input('btn-all-years', 'n_clicks'),
    prevent_initial_call=True
)
def reset_date_filters(n_clicks):
    if n_clicks:
        # Reset to the actual data range in your dataset
        return datetime(2022, 4, 18), datetime(2025, 4, 24)
    return datetime(2022, 4, 1), datetime(2025, 4, 24)

print("✅ All callbacks configured with your actual data structure!")
print(f"📊 Dashboard ready for {len(df)} students with real metrics!")
print("🔄 Charts will now show EMPTY when no data matches filters!")
print("⏰ 'All Time' button will reset date range automatically!")

🔗 Creating callbacks to match template with your actual data...
✅ All callbacks configured with your actual data structure!
📊 Dashboard ready for 285 students with real metrics!
🔄 Charts will now show EMPTY when no data matches filters!
⏰ 'All Time' button will reset date range automatically!


In [34]:
# === DATA PREVIEW FOR DASHBOARD ===
print("📋 Your Student Data Preview:")
print("=" * 50)
print(f"📊 Total Students: {len(df)}")
print(f"🎯 Sample Student IDs: {list(df['STUDENT ID'].head(3))}")
print(f"📈 GPA Range: {df['CUMULATIVE_GPA'].min():.2f} - {df['CUMULATIVE_GPA'].max():.2f}")
print(f"⏱️ Completion Time Range: {df['COMPLETION_MONTHS'].min():.1f} - {df['COMPLETION_MONTHS'].max():.1f} months")
print(f"🎓 Pass Rate: {((df['CUMULATIVE_GPA'] >= 2.0).mean() * 100):.1f}%")

print("\n🔍 Dashboard Search Tips:")
print("- Search by partial Student ID (e.g., 'STU', '001', etc.)")
print("- Use date filters to narrow by commencement period")
print("- KPIs update automatically based on filters")
print("=" * 50)

📋 Your Student Data Preview:
📊 Total Students: 285
🎯 Sample Student IDs: ['1101-009/001', '1101-009/002', '1101-009/003']
📈 GPA Range: 2.00 - 4.00
⏱️ Completion Time Range: 1.0 - 23.0 months
🎓 Pass Rate: 95.4%

🔍 Dashboard Search Tips:
- Search by partial Student ID (e.g., 'STU', '001', etc.)
- Use date filters to narrow by commencement period
- KPIs update automatically based on filters


In [35]:
# === TEST CHART UPDATES ===
print("🔬 Testing chart update logic...")

# Check available citizenship values
print(f"📋 Available citizenship types: {df['CITIZENSHIP'].unique()}")
print(f"📊 Citizenship counts: \n{df['CITIZENSHIP'].value_counts()}")

# Test with actual data - use first available citizenship type
first_citizenship = df['CITIZENSHIP'].value_counts().index[0]
test_filter = df[df['CITIZENSHIP'] == first_citizenship].head(50)
print(f"\n📊 Test data: {len(test_filter)} students ({first_citizenship} only)")

# Test treemap generation
treemap_test = test_filter.groupby(['CITIZENSHIP', 'GENDER', 'COURSE FUNDING']).size().reset_index(name='COUNT')
print(f"🎯 Treemap categories: {len(treemap_test)}")

# Test scatter data
scatter_test = test_filter.dropna(subset=['CUMULATIVE_GPA', 'COMPLETION_MONTHS'])
print(f"📈 Scatter plot points: {len(scatter_test)}")

# Test box plot data
box_test = test_filter.dropna(subset=['COMPLETION_MONTHS'])
box_test = box_test[box_test['COMPLETION_MONTHS'] > 0]
print(f"📦 Box plot data points: {len(box_test)}")

print("\n✅ Chart update logic verification complete!")
print("🚀 Charts will now update dynamically when you apply filters in the dashboard!")
print(f"💡 Try searching for student IDs containing: {list(df['STUDENT ID'].head(3))}")

🔬 Testing chart update logic...
📋 Available citizenship types: ['FOREIGNER' 'SG CITIZEN' 'SG PR']
📊 Citizenship counts: 
CITIZENSHIP
SG CITIZEN    221
FOREIGNER      34
SG PR          30
Name: count, dtype: int64

📊 Test data: 50 students (SG CITIZEN only)
🎯 Treemap categories: 5
📈 Scatter plot points: 50
📦 Box plot data points: 50

✅ Chart update logic verification complete!
🚀 Charts will now update dynamically when you apply filters in the dashboard!
💡 Try searching for student IDs containing: ['1101-009/001', '1101-009/002', '1101-009/003']


In [36]:
# === DEBUG CHART UPDATE ISSUE ===
print("🔍 Debugging chart update issue...")

# Test date filtering logic
print(f"📅 COMMENCEMENT DATE column exists: {'COMMENCEMENT DATE' in df.columns}")
print(f"📅 Date range in data: {df['COMMENCEMENT DATE'].min()} to {df['COMMENCEMENT DATE'].max()}")

# Test with your actual date range
start_test = pd.to_datetime('2020-01-01')
end_test = pd.to_datetime('2021-12-31')

print(f"\n🔬 Testing filter: {start_test} to {end_test}")
test_filtered = df.copy()
test_filtered = test_filtered[pd.to_datetime(test_filtered['COMMENCEMENT DATE']) >= start_test]
test_filtered = test_filtered[pd.to_datetime(test_filtered['COMMENCEMENT DATE']) <= end_test]

print(f"📊 Original data: {len(df)} students")
print(f"📊 Filtered data: {len(test_filtered)} students")
print(f"📊 Date distribution in filtered data:")
print(test_filtered['COMMENCEMENT DATE'].dt.year.value_counts().sort_index())

# Test another date range
start_test2 = pd.to_datetime('2020-01-01') 
end_test2 = pd.to_datetime('2020-12-31')
test_filtered2 = df.copy()
test_filtered2 = test_filtered2[pd.to_datetime(test_filtered2['COMMENCEMENT DATE']) >= start_test2]
test_filtered2 = test_filtered2[pd.to_datetime(test_filtered2['COMMENCEMENT DATE']) <= end_test2]

print(f"\n🔬 Testing 2020 only: {start_test2} to {end_test2}")
print(f"📊 2020 only filtered: {len(test_filtered2)} students")

print("\n💡 If charts aren't updating, the issue might be:")
print("   1. Callback errors (check browser console)")
print("   2. Data conversion issues in callbacks") 
print("   3. Missing imports in callback functions")

🔍 Debugging chart update issue...
📅 COMMENCEMENT DATE column exists: True
📅 Date range in data: 2022-04-18 00:00:00 to 2025-04-24 00:00:00

🔬 Testing filter: 2020-01-01 00:00:00 to 2021-12-31 00:00:00
📊 Original data: 285 students
📊 Filtered data: 0 students
📊 Date distribution in filtered data:
Series([], Name: count, dtype: int64)

🔬 Testing 2020 only: 2020-01-01 00:00:00 to 2020-12-31 00:00:00
📊 2020 only filtered: 0 students

💡 If charts aren't updating, the issue might be:
   1. Callback errors (check browser console)
   2. Data conversion issues in callbacks
   3. Missing imports in callback functions


In [37]:
# === LAUNCH DASHBOARD ===
if __name__ == '__main__':
    
    print("🌟 Dashboard will be available at: http://127.0.0.1:8052/")
   
    
    app.run(debug=True, port=8052)

🌟 Dashboard will be available at: http://127.0.0.1:8052/


### XAVIER LEE ZHUO MIN

In [38]:
#!/usr/bin/env python3
"""
XAVIER LEE ZHUO MIN - SINGAPORE STUDENT PATHWAYS DASHBOARD
=========================================================
Interactive dashboard with futuristic styling and all filters at the top

DASHBOARD FEATURES:
- Real-time student data visualization
- Interactive filtering (date, course, age group)
- 4 main charts: Sunburst, Line, Violin, Scatter
- Student search functionality  
- Responsive KPI cards
- Futuristic neon styling with animations
"""

# ================================
# LIBRARY IMPORTS
# ================================
import dash                              # Main web framework for dashboard
from dash import dcc, html, Input, Output, callback  # Dashboard components and callbacks
import dash_bootstrap_components as dbc  # Bootstrap styling components
import pandas as pd                      # Data manipulation and analysis
import numpy as np                       # Numerical computations
from pathlib import Path                 # File path handling
from datetime import datetime            # Date and time operations
import plotly.express as px             # High-level plotting interface
import plotly.graph_objects as go        # Low-level plotting for custom charts

# ================================
# DATA LOADING & PREPROCESSING SECTION
# ================================

# Data directory setup - points to folder containing cleaned Excel files
DATA_DIR = Path.cwd() / "DAVI CA2 datasets and meta data"

def load_data():
    """
    LOAD THREE MAIN DATASETS
    ========================
    Loads the cleaned Excel files containing:
    - Student profiles (demographic data)
    - Semester results (GPA records) 
    - Course codes (course name mappings)
    
    Returns:
        tuple: (profiles_df, semester_df, codes_df)
    """
    profiles = pd.read_excel("Cleaned_Student_Profiles.xlsx")
    sem = pd.read_excel("Cleaned_Semester_Results.xlsx")
    codes = pd.read_excel("Cleaned_Course_Codes.xlsx")
    return profiles, sem, codes

def derive_level(course_name: str) -> str:
    """
    COURSE LEVEL CATEGORIZATION
    ===========================
    Automatically categorizes courses based on name patterns:
    - "Specialist Diploma" → Specialist Diploma
    - "Diploma" (not specialist) → Diploma  
    - "Certificate" → Certificate
    - Others → Other
    
    Args:
        course_name (str): Full course name
        
    Returns:
        str: Course level category
    """
    if isinstance(course_name, str):
        s = course_name.lower()
        if s.startswith("specialist diploma"):
            return "Specialist Diploma"
        if s.startswith("diploma"):
            return "Diploma"
        if s.startswith("certificate"):
            return "Certificate"
    return "Other"

def preprocess(profiles: pd.DataFrame, sem: pd.DataFrame, codes: pd.DataFrame):
    """
    COMPREHENSIVE DATA PREPROCESSING PIPELINE
    ========================================
    Transforms raw data into dashboard-ready format by:
    
    1. DATE PROCESSING: Converts string dates to datetime objects
    2. DATA MERGING: Links course codes with descriptive names
    3. DERIVED FIELDS: Creates calculated columns for analysis
    4. DATA CLEANING: Standardizes categorical values
    5. RELATIONSHIP BUILDING: Connects profiles with semester data
    
    Args:
        profiles (DataFrame): Student demographic data
        sem (DataFrame): Semester academic results  
        codes (DataFrame): Course code to name mappings
        
    Returns:
        dict: Contains processed dataframes {profiles, sem, codes}
    """
    
    # STEP 1: DATE FIELD CONVERSION
    # Convert string dates to pandas datetime for proper filtering
    for col in ["DOB", "COMMENCEMENT DATE", "COMPLETION DATE", "DATE ATTAINED HIGHEST QUALIFICATION"]:
        if col in profiles.columns:
            profiles[col] = pd.to_datetime(profiles[col], errors="coerce")

    # STEP 2: COURSE NAME INTEGRATION
    # Merge course codes with descriptive names for better visualization
    code_map = codes.rename(columns={"CODE": "COURSE CODE", "COURSE NAME": "COURSE NAME"})
    profiles = profiles.merge(code_map, on="COURSE CODE", how="left")

    # STEP 3: DERIVED FIELD CREATION
    # Create new analytical columns from existing data
    profiles["COURSE LEVEL"] = profiles["COURSE NAME"].apply(derive_level)
    profiles["DesignationStatus"] = profiles["DESIGNATION"].fillna("Unemployed").replace("", "Unemployed")
    profiles["EmployedFlag"] = profiles["DesignationStatus"].ne("Unemployed")  # Boolean employment status
    profiles["duration_days"] = (profiles["COMPLETION DATE"] - profiles["COMMENCEMENT DATE"]).dt.days
    
    # STEP 4: CATEGORICAL DATA STANDARDIZATION
    # Standardize citizenship categories for consistent visualization
    profiles["CitizenshipSimple"] = profiles["CITIZENSHIP"].replace({
        "SG CITIZEN": "SG Citizen",
        "SG PR": "SG PR",
        "FOREIGNER": "Foreigner"
    })
    # Standardize gender labels
    profiles["Gender"] = profiles["GENDER"].replace({"F":"Female", "M":"Male"})
    # Standardize study mode labels
    profiles["FT_PT"] = profiles["FULL-TIME OR PART-TIME"].replace({"Part Time":"Part-Time"})

    # STEP 5: SEMESTER DATA ENHANCEMENT
    # Add course information and demographics to semester records
    sem = sem.merge(code_map, on="COURSE CODE", how="left")
    sem["COURSE LEVEL"] = sem["COURSE NAME"].apply(derive_level)
    sem["PERIOD_NUM"] = sem["PERIOD"].str.extract(r"(\d+)").astype(float)  # Extract semester numbers
    
    # Link semester data with student demographics for filtering
    sem = sem.merge(
        profiles[["STUDENT ID","CitizenshipSimple","Gender","COURSE CODE"]], 
        on=["STUDENT ID","COURSE CODE"], 
        how="left"
    )
    
    return {"profiles": profiles, "sem": sem, "codes": codes}

# ================================
# DATA LOADING EXECUTION
# ================================

# Load and preprocess all data for dashboard use
print("🚀 Loading Singapore student pathway data...")
profiles_raw, sem_raw, codes = load_data()
data = preprocess(profiles_raw, sem_raw, codes)

# Extract processed dataframes for easy access
profiles = data["profiles"]  # Main student demographic data
sem = data["sem"]           # Semester academic performance data

print(f"✅ Successfully loaded:")
print(f"   📊 {len(profiles)} student profiles")
print(f"   📈 {len(sem)} semester records")
print(f"   📚 {len(codes)} course codes")

# ================================
# FUTURISTIC COLOR SCHEME & STYLING
# ================================

# XAVIER'S CUSTOM NEON COLOR PALETTE
# Carefully chosen colors for maximum visual impact and readability
COLORS = {
    # === BACKGROUND COLORS ===
    'dark_bg': '#0A0A1A',        # Deep space black - main background
    'card_bg': '#1A1A2E',        # Dark navy - card backgrounds  
    'sidebar_bg': '#16213E',     # Midnight blue - sidebar backgrounds
    
    # === NEON ACCENT COLORS ===
    'neon_blue': '#00F5FF',      # Electric cyan - primary highlights
    'neon_purple': '#C77DFF',    # Bright purple - secondary highlights
    'neon_green': '#22FF22',     # Vivid green - success indicators
    'accent': '#FF6B9D',         # Hot pink - special accents
    'pink': '#FF1493',           # Deep pink - alerts
    
    # === STATUS INDICATOR COLORS ===
    'primary': '#00F5FF',        # Primary actions (cyan)
    'secondary': '#C77DFF',      # Secondary actions (purple)
    'success': '#22FF22',        # Success states (green)
    'warning': '#FFD700',        # Warning states (gold)
    'danger': '#FF073A',         # Error states (red)
    'info': '#17A2B8',          # Information (teal)
    
    # === TEXT COLORS ===
    'text': '#FFFFFF',           # Primary text (white)
    'text_dim': '#B0BEC5',       # Secondary text (light gray)
    'light': '#F8F9FA',          # Light accents
    
    # === SPECIAL EFFECTS ===
    'glow_shadow': 'rgba(0, 245, 255, 0.4)'  # Glowing shadow effect
}

# ================================
# DASH APPLICATION INITIALIZATION
# ================================

# Initialize Dash app with Bootstrap theme for responsive design
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.title = "Singapore Student Pathways Dashboard"  # Browser tab title

# ================================
# UI COMPONENT DEFINITIONS
# ================================

# FUTURISTIC HEADER COMPONENT
# Creates the main dashboard title with animated neon effects
header = html.Div([
    html.Div([
        # Main title with glowing animation
        html.H1("SINGAPORE STUDENT PATHWAYS", 
               style={
                   'color': COLORS['neon_blue'], 'margin': '0', 'fontWeight': '900',
                   'fontSize': '3rem', 'letterSpacing': '3px', 'textAlign': 'center',
                   'fontFamily': 'Orbitron, monospace',  # Futuristic font
                   'textShadow': f'0 0 20px {COLORS["neon_blue"]}, 0 0 40px {COLORS["neon_blue"]}60',
                   'animation': 'pulse 2s infinite'  # CSS animation defined in index_string
               }),
        # Subtitle with accent color
        html.P("INTERACTIVE ANALYTICS DASHBOARD", 
              style={
                  'color': COLORS['accent'], 'margin': '10px 0 0 0', 'textAlign': 'center',
                  'fontSize': '1.2rem', 'fontWeight': '600', 'letterSpacing': '1px',
                  'fontFamily': 'Rajdhani, monospace',
                  'textShadow': f'0 0 10px {COLORS["accent"]}'
              })
    ], style={'textAlign': 'center', 'padding': '20px'})
], style={
    # Gradient background with glowing border
    'background': f'linear-gradient(135deg, {COLORS["dark_bg"]} 0%, {COLORS["card_bg"]} 50%, {COLORS["dark_bg"]} 100%)',
    'borderBottom': f'3px solid {COLORS["neon_blue"]}',
    'boxShadow': f'0 5px 20px {COLORS["glow_shadow"]}',
    'marginBottom': '0'
})

# ================================
# REUSABLE STYLING DEFINITIONS  
# ================================

# CHART CONTAINER STYLING
# Standard styling for all chart containers with neon glow effects
chart_style = {
    'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)',
    'border': f'2px solid {COLORS["neon_blue"]}40',  # Semi-transparent border
    'borderRadius': '25px',
    'padding': '30px',
    'marginBottom': '30px',
    'boxShadow': f'0 0 40px {COLORS["glow_shadow"]}, 0 15px 60px rgba(26, 26, 46, 0.8)',
    'backdropFilter': 'blur(15px)',  # Glassmorphism effect
    'position': 'relative',
    'zIndex': '1',
    'overflow': 'hidden',
    'transition': 'all 0.3s ease'  # Smooth hover transitions
}

# Charts section
charts_section = dbc.Container([
    # Row 1: Sunburst and Line Chart
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("CITIZENSHIP DISTRIBUTION", 
                       style={
                           'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                           'textTransform': 'uppercase'
                       }),
                dcc.Graph(id='sunburst-chart', config={'displayModeBar': False})
            ], style=chart_style)
        ], width=4),  # Made smaller (was 6)
        
        dbc.Col([
            html.Div([
                html.H4("GPA TRENDS BY SEMESTER", 
                       style={
                           'color': COLORS['neon_purple'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["neon_purple"]}, 0 0 30px {COLORS["neon_purple"]}40',
                           'textTransform': 'uppercase'
                       }),
                dcc.Graph(id='gpa-trend-chart', config={'displayModeBar': False})
            ], style=chart_style)
        ], width=8)  # Made wider (was 6)
    ], className="g-4"),
    
    # Row 2: Violin and Scatter plots
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("AGE DISTRIBUTION ANALYSIS", 
                       style={
                           'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                           'textTransform': 'uppercase'
                       }),
                dcc.Graph(id='violin-chart', config={'displayModeBar': False})
            ], style=chart_style)
        ], width=4),  # Made smaller (was 6)
        
        dbc.Col([
            html.Div([
                html.H4("COURSE POPULARITY BY GENDER", 
                       style={
                           'color': COLORS['accent'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["accent"]}, 0 0 30px {COLORS["accent"]}40',
                           'textTransform': 'uppercase'
                       }),
                dcc.Graph(id='scatter-chart', config={'displayModeBar': False})
            ], style=chart_style)
        ], width=8)  # Made wider (was 6)
    ], className="g-4"),
    
    # Student data table section
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("STUDENT DATA SEARCH RESULTS", 
                       style={
                           'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '25px',
                           'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                           'fontFamily': 'Orbitron, monospace',
                           'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                           'textTransform': 'uppercase'
                       }),
                html.Div(id='student-data-table')
            ], style=chart_style)
        ], width=12)
    ], className="g-4")
], fluid=True)

# ================================
# LOADING SCREEN COMPONENT
# ================================

# Loading screen overlay component
loading_overlay = html.Div([
    html.Div([
        # Holographic spinner
        html.Div(className="holo-spinner"),
        
        # Glitch loading text
        html.Div("LOADING DASHBOARD...", className="glitch-loader", style={'marginTop': '30px'}),
        
        # Optional subtitle
        html.P("Initializing neural pathways...", style={
            'color': COLORS['text_dim'],
            'fontFamily': 'Rajdhani, monospace',
            'fontSize': '0.9rem',
            'marginTop': '20px',
            'animation': 'fadeInOut 2s infinite'
        })
    ], style={
        'textAlign': 'center',
        'padding': '50px'
    })
], id='loading-overlay', style={
    'position': 'fixed',
    'top': '0',
    'left': '0',
    'width': '100%',
    'height': '100%',
    'backgroundColor': 'rgba(10, 10, 26, 0.95)',
    'zIndex': '9999',
    'display': 'flex',
    'alignItems': 'center',
    'justifyContent': 'center',
    'backdropFilter': 'blur(10px)'
})

# ================================
# MAIN DASHBOARD LAYOUT STRUCTURE
# ================================

app.layout = html.Div([
    # LOADING OVERLAY - Shows while dashboard initializes
    loading_overlay,
    
    # MAIN DASHBOARD CONTENT
    html.Div([
        # GOOGLE FONTS INTEGRATION - Load futuristic fonts
        html.Link(
            rel='stylesheet',
            href='https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&family:Rajdhani:wght@300;400;500;600;700&family=Space+Mono:wght@400;700&display=swap'
        ),
    
    # HEADER SECTION (Full Width)
    header,
    
    # MAIN CONTENT CONTAINER - Two-column responsive layout
    dbc.Container([
        dbc.Row([
            # =====================================
            # LEFT SIDEBAR (4/12 columns) - CONTROLS & INFO
            # =====================================
            dbc.Col([
                # DATE FILTER SECTION
                # Allows users to filter data by course commencement dates
                html.Div([
                    html.H4("📅 DATE FILTER", 
                           style={
                               'color': COLORS['neon_purple'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_purple"]}, 0 0 30px {COLORS["neon_purple"]}40',
                               'textTransform': 'uppercase'
                           }),
                    
                    # Interactive date range picker
                    html.Div([
                        html.Label("Course Commencement Period", 
                                  style={'color': COLORS['neon_purple'], 'fontWeight': 'bold', 'marginBottom': '10px'}),
                        dcc.DatePickerRange(
                            id='date-range-picker',
                            start_date=profiles['COMMENCEMENT DATE'].min(),  # Auto-set to data range
                            end_date=profiles['COMMENCEMENT DATE'].max(),
                            display_format='DD/MM/YYYY',
                            style={
                                'fontFamily': 'Rajdhani, monospace',
                                'fontWeight': '600'
                            },
                            className='custom-date-picker'
                        )
                    ], style={'marginBottom': '20px'}),
                    
                    # Quick preset buttons for common date ranges
                    html.Div([
                        html.Label("Quick Date Presets", 
                                  style={'color': COLORS['accent'], 'fontWeight': 'bold', 'marginBottom': '10px'}),
                        dcc.RadioItems(
                            id='date-preset-radio',
                            options=[
                                {'label': 'All Time', 'value': 'all'},
                                {'label': 'Last 2 Years', 'value': 'last_2_years'},
                                {'label': 'Last 5 Years', 'value': 'last_5_years'},
                                {'label': '2020 and After', 'value': 'since_2020'}
                            ],
                            value='all',
                            inline=False,
                            style={'color': COLORS['text']},
                            className='custom-radio',
                            inputStyle={
                                'marginRight': '8px',
                                'accentColor': COLORS['accent']
                            },
                            labelStyle={
                                'display': 'block',
                                'marginBottom': '8px',
                                'padding': '8px 12px',
                                'backgroundColor': COLORS['sidebar_bg'],
                                'border': f'2px solid {COLORS["accent"]}40',
                                'borderRadius': '8px',
                                'cursor': 'pointer',
                                'transition': 'all 0.3s ease',
                                'fontFamily': 'Rajdhani, monospace',
                                'fontWeight': '500'
                            }
                        )
                    ], style={'marginBottom': '10px'})
                ], style={**chart_style, 'marginBottom': '20px'}),
                
                # Filter Controls Section
                html.Div([
                    html.H4("🎛️ FILTER CONTROLS", 
                           style={
                               'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                               'textTransform': 'uppercase'
                           }),
                    
                    # Course Selection
                    html.Div([
                        html.Label("Course Selection (Line Chart)", 
                                  style={'color': COLORS['neon_blue'], 'fontWeight': 'bold', 'marginBottom': '10px'}),
                        dcc.Dropdown(
                            id='course-dropdown',
                            placeholder="Select course for GPA trend analysis",
                            style={
                                'backgroundColor': COLORS['dark_bg'],
                                'color': COLORS['text'],
                                'fontFamily': 'Rajdhani, monospace',
                                'fontWeight': '600',
                                'border': f'1px solid {COLORS["neon_blue"]}',
                                'borderRadius': '5px'
                            },
                            className='custom-dropdown',
                            optionHeight=40,
                            clearable=False
                        )
                    ], style={'marginBottom': '20px'}),
                    
                    # Course Level Filter
                    html.Div([
                        html.Label("Course Level Filter (Scatter Plot)", 
                                  style={'color': COLORS['accent'], 'fontWeight': 'bold', 'marginBottom': '10px'}),
                        dcc.Dropdown(
                            id='course-level-dropdown',
                            options=[
                                {'label': html.Span('All Course Levels', style={'color': COLORS['text'], 'fontFamily': 'Rajdhani, monospace', 'fontWeight': '600'}), 'value': 'all'},
                                {'label': html.Span('Certificate Courses Only', style={'color': COLORS['text'], 'fontFamily': 'Rajdhani, monospace', 'fontWeight': '600'}), 'value': 'Certificate'},
                                {'label': html.Span('Diploma Courses Only', style={'color': COLORS['text'], 'fontFamily': 'Rajdhani, monospace', 'fontWeight': '600'}), 'value': 'Diploma'},
                                {'label': html.Span('Specialist Diploma Courses Only', style={'color': COLORS['text'], 'fontFamily': 'Rajdhani, monospace', 'fontWeight': '600'}), 'value': 'Specialist Diploma'}
                            ],
                            value='all',
                            style={
                                'backgroundColor': COLORS['dark_bg'],
                                'color': COLORS['text'],
                                'fontFamily': 'Rajdhani, monospace',
                                'fontWeight': '600',
                                'border': f'1px solid {COLORS["neon_blue"]}',
                                'borderRadius': '5px'
                            },
                            className='custom-dropdown',
                            optionHeight=40,
                            clearable=False
                        )
                    ], style={'marginBottom': '20px'}),
                    
                    # Age Group Filter
                    html.Div([
                        html.Label("Age Group Filter (Violin Plot)", 
                                  style={'color': COLORS['neon_purple'], 'fontWeight': 'bold', 'marginBottom': '10px'}),
                        dcc.RadioItems(
                            id='age-group-radio',
                            options=[
                                {'label': 'All Ages (15-80)', 'value': 'all'},
                                {'label': 'Young Adults (15-25)', 'value': 'young'},
                                {'label': 'Adults (26-40)', 'value': 'adult'},
                                {'label': 'Mature Adults (41-80)', 'value': 'mature'}
                            ],
                            value='all',
                            inline=False,
                            style={'color': COLORS['text']},
                            className='custom-radio',
                            inputStyle={
                                'marginRight': '8px',
                                'accentColor': COLORS['neon_purple']
                            },
                            labelStyle={
                                'display': 'block',
                                'marginBottom': '8px',
                                'padding': '8px 12px',
                                'backgroundColor': COLORS['sidebar_bg'],
                                'border': f'2px solid {COLORS["neon_purple"]}40',
                                'borderRadius': '8px',
                                'cursor': 'pointer',
                                'transition': 'all 0.3s ease',
                                'fontFamily': 'Rajdhani, monospace',
                                'fontWeight': '500'
                            }
                        )
                    ], style={'marginBottom': '20px'})
                ], style={**chart_style, 'marginBottom': '20px'}),
                
                # Search Section
                html.Div([
                    html.Label("🔍 SEARCH STUDENT ID", style={
                        'color': COLORS['neon_green'], 'fontWeight': '700', 'marginBottom': '15px',
                        'fontSize': '1rem', 'letterSpacing': '1px',
                        'textShadow': f'0 0 8px {COLORS["neon_green"]}',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    dcc.Input(
                        id='search-box',
                        type='text',
                        placeholder='Search by Student ID...',
                        style={
                            'width': '100%',
                            'backgroundColor': COLORS['sidebar_bg'],
                            'color': COLORS['text'],
                            'border': f'2px solid {COLORS["neon_green"]}',
                            'borderRadius': '8px',
                            'padding': '10px 12px',
                            'fontSize': '14px',
                            'fontFamily': 'Rajdhani, monospace',
                            'fontWeight': '500'
                        }
                    )
                ], style={
                    'backgroundColor': COLORS['card_bg'], 'padding': '20px', 'borderRadius': '20px',
                    'boxShadow': f'0 0 30px {COLORS["neon_green"]}40, 0 8px 32px rgba(26, 26, 46, 0.6)', 
                    'border': f'2px solid {COLORS["neon_green"]}40',
                    'backdropFilter': 'blur(10px)',
                    'marginBottom': '20px'
                }),
                
                # Student Data Section
                html.Div([
                    html.H4("📊 STUDENT DATA", 
                           style={
                               'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                               'textTransform': 'uppercase'
                           }),
                    html.Div(id='student-data-table')
                ], style={**chart_style, 'height': '300px', 'overflow': 'auto', 'marginBottom': '20px'})
                
            ], width=4),  # Left side takes 4/12 columns
            
            # RIGHT SIDE - Scrollable Charts (8/12 columns)
            dbc.Col([
                # Scrollable Charts Container
                html.Div([
                    # KPI Section at the top
                    html.Div([
                        html.H4("📈 KEY METRICS", 
                               style={
                                   'color': COLORS['neon_purple'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_purple"]}, 0 0 30px {COLORS["neon_purple"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        html.Div(id='kpi-cards', style={
                            'display': 'grid',
                            'gridTemplateColumns': 'repeat(auto-fit, minmax(200px, 1fr))',
                            'gap': '20px',
                            'padding': '0 10px'
                        })
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 1 - Sunburst Chart
                    html.Div([
                        html.H4("CITIZENSHIP DISTRIBUTION", 
                               style={
                                   'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='sunburst-chart', 
                                 config={'displayModeBar': False},
                                 style={'height': '600px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 2 - GPA Trends
                    html.Div([
                        html.H4("GPA TRENDS BY SEMESTER", 
                               style={
                                   'color': COLORS['neon_purple'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_purple"]}, 0 0 30px {COLORS["neon_purple"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='gpa-trend-chart', 
                                 config={'displayModeBar': False},
                                 style={'height': '600px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 3 - Age Distribution
                    html.Div([
                        html.H4("AGE DISTRIBUTION ANALYSIS", 
                               style={
                                   'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='violin-chart', 
                                 config={'displayModeBar': False},
                                 style={'height': '600px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 4 - Course Popularity
                    html.Div([
                        html.H4("COURSE POPULARITY BY GENDER", 
                               style={
                                   'color': COLORS['accent'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["accent"]}, 0 0 30px {COLORS["accent"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='scatter-chart', 
                                 config={'displayModeBar': False},
                                 style={'height': '600px'})
                    ], style=chart_style)
                    
                ], style={
                    'height': '165vh',  # Set viewport height
                    'overflowY': 'auto',  # Enable vertical scrolling
                    'paddingRight': '15px',  # Space for scrollbar
                    'scrollbarWidth': 'thin',  # Thin scrollbar for Firefox
                    'scrollbarColor': f'{COLORS["neon_blue"]} {COLORS["sidebar_bg"]}'  # Custom scrollbar colors
                })
                
            ], width=8)  # Right side takes 8/12 columns
            
        ], className="g-4")
    ], fluid=True)
    ], id='main-content', style={
        'backgroundColor': COLORS['dark_bg'], 
        'minHeight': '100vh', 
        'padding': '0',
        'fontFamily': 'Rajdhani, Orbitron, monospace'
    })
], style={
    'backgroundColor': COLORS['dark_bg'], 
    'minHeight': '100vh'
})

# Loading screen overlay component
loading_overlay = html.Div([
    html.Div([
        # Holographic spinner
        html.Div(className="holo-spinner"),
        
        # Glitch loading text
        html.Div("LOADING DASHBOARD...", className="glitch-loader", style={'marginTop': '30px'}),
        
        # Optional subtitle
        html.P("Initializing neural pathways...", style={
            'color': COLORS['text_dim'],
            'fontFamily': 'Rajdhani, monospace',
            'fontSize': '0.9rem',
            'marginTop': '20px',
            'animation': 'fadeInOut 2s infinite'
        })
    ], style={
        'textAlign': 'center',
        'padding': '50px'
    })
], id='loading-overlay', style={
    'position': 'fixed',
    'top': '0',
    'left': '0',
    'width': '100%',
    'height': '100%',
    'backgroundColor': 'rgba(10, 10, 26, 0.95)',
    'zIndex': '9999',
    'display': 'flex',
    'alignItems': 'center',
    'justifyContent': 'center',
    'backdropFilter': 'blur(10px)'
})

# Add custom CSS for enhanced styling through app.index_string
app.index_string = f'''
<!DOCTYPE html>
<html>
    <head>
        {{%metas%}}
        <title>{{%title%}}</title>
        {{%favicon%}}
        {{%css%}}
        <style>
            /* Loading Screen Animations */
            .holo-spinner {{
                width: 80px;
                height: 80px;
                border: 3px solid transparent;
                border-top: 3px solid {COLORS['neon_blue']};
                border-right: 3px solid {COLORS['neon_purple']};
                border-radius: 50%;
                position: relative;
                animation: holoSpin 1s linear infinite;
                margin: 0 auto;
                box-shadow: 0 0 20px {COLORS['neon_blue']}60;
            }}

            .holo-spinner::before {{
                content: '';
                position: absolute;
                top: -3px;
                left: -3px;
                right: -3px;
                bottom: -3px;
                border: 3px solid transparent;
                border-bottom: 3px solid {COLORS['neon_green']};
                border-left: 3px solid {COLORS['accent']};
                border-radius: 50%;
                animation: holoSpin 2s linear infinite reverse;
            }}

            @keyframes holoSpin {{
                0% {{ transform: rotate(0deg); }}
                100% {{ transform: rotate(360deg); }}
            }}

            .glitch-loader {{
                font-family: 'Orbitron', monospace;
                font-size: 2rem;
                color: {COLORS['neon_blue']};
                position: relative;
                animation: glitch 2s infinite;
                font-weight: 900;
                letter-spacing: 3px;
                text-shadow: 0 0 20px {COLORS['neon_blue']};
            }}

            .glitch-loader::before,
            .glitch-loader::after {{
                content: 'LOADING DASHBOARD...';
                position: absolute;
                top: 0;
                left: 0;
                opacity: 0.8;
            }}

            .glitch-loader::before {{
                color: {COLORS['accent']};
                animation: glitch-1 0.3s infinite linear alternate-reverse;
            }}

            .glitch-loader::after {{
                color: {COLORS['neon_green']};
                animation: glitch-2 0.3s infinite linear alternate-reverse;
            }}

            @keyframes glitch {{
                0% {{ transform: translateX(0); }}
                20% {{ transform: translateX(-2px); }}
                40% {{ transform: translateX(2px); }}
                60% {{ transform: translateX(-2px); }}
                80% {{ transform: translateX(2px); }}
                100% {{ transform: translateX(0); }}
            }}

            @keyframes glitch-1 {{
                0% {{ clip-path: polygon(0 0%, 100% 0%, 100% 5%, 0 5%); }}
                100% {{ clip-path: polygon(0 85%, 100% 85%, 100% 100%, 0 100%); }}
            }}

            @keyframes glitch-2 {{
                0% {{ clip-path: polygon(0 15%, 100% 15%, 100% 30%, 0 30%); }}
                100% {{ clip-path: polygon(0 60%, 100% 60%, 100% 70%, 0 70%); }}
            }}

            @keyframes fadeInOut {{
                0%, 100% {{ opacity: 0.5; }}
                50% {{ opacity: 1; }}
            }}

            /* Custom scrollbar styling for chart container */
            ::-webkit-scrollbar {{
                width: 12px;
            }}
            ::-webkit-scrollbar-track {{
                background: {COLORS['card_bg']};
                border-radius: 10px;
            }}
            ::-webkit-scrollbar-thumb {{
                background: linear-gradient(135deg, {COLORS['neon_blue']}, {COLORS['neon_purple']});
                border-radius: 10px;
                border: 2px solid {COLORS['card_bg']};
            }}
            ::-webkit-scrollbar-thumb:hover {{
                background: linear-gradient(135deg, {COLORS['neon_green']}, {COLORS['accent']});
                box-shadow: 0 0 10px {COLORS['neon_blue']}60;
            }}
            
            /* Force all dropdown elements to be visible */
            [id*="dropdown"] *, [class*="dropdown"] *,
            div[data-dash-is-loading] *, .dash-dropdown * {{
                color: {COLORS['text']} !important;
                font-family: 'Rajdhani', monospace !important;
                font-weight: 600 !important;
                background-color: {COLORS['sidebar_bg']} !important;
            }}
            
            /* Specific targeting for dropdown controls */
            .custom-dropdown > div, .custom-dropdown > div > div,
            .custom-dropdown > div > div > div {{
                background-color: {COLORS['sidebar_bg']} !important;
                color: {COLORS['text']} !important;
                border: 1px solid {COLORS['neon_blue']} !important;
                border-radius: 5px !important;
                font-family: 'Rajdhani', monospace !important;
                font-weight: 600 !important;
                min-height: 38px !important;
            }}
            
            /* Override any React-Select default styling */
            .Select-control, .Select-placeholder, .Select-value-label,
            .Select-menu-outer, .Select-option {{
                background-color: {COLORS['sidebar_bg']} !important;
                color: {COLORS['text']} !important;
                font-family: 'Rajdhani', monospace !important;
                font-weight: 600 !important;
            }}
            
            /* Enhanced radio button styling */
            .custom-radio input[type="radio"] {{
                appearance: none !important;
                width: 18px !important;
                height: 18px !important;
                border: 2px solid {COLORS['neon_purple']} !important;
                border-radius: 50% !important;
                background-color: transparent !important;
                position: relative !important;
                margin-right: 8px !important;
                cursor: pointer !important;
            }}
            
            .custom-radio input[type="radio"]:checked {{
                background-color: {COLORS['neon_purple']} !important;
                box-shadow: 0 0 10px {COLORS['neon_purple']}60 !important;
            }}
            
            .custom-radio input[type="radio"]:checked::after {{
                content: '' !important;
                position: absolute !important;
                top: 50% !important;
                left: 50% !important;
                transform: translate(-50%, -50%) !important;
                width: 8px !important;
                height: 8px !important;
                border-radius: 50% !important;
                background-color: white !important;
            }}
            
            .custom-radio label {{
                display: flex !important;
                align-items: center !important;
                margin-bottom: 8px !important;
                padding: 10px 14px !important;
                background-color: {COLORS['sidebar_bg']} !important;
                border: 2px solid {COLORS['neon_purple']}40 !important;
                border-radius: 8px !important;
                cursor: pointer !important;
                transition: all 0.3s ease !important;
                font-family: 'Rajdhani', monospace !important;
                font-weight: 500 !important;
                color: {COLORS['text']} !important;
            }}
            
            .custom-radio label:hover {{
                border-color: {COLORS['neon_purple']} !important;
                background-color: {COLORS['neon_purple']}10 !important;
                box-shadow: 0 0 12px {COLORS['neon_purple']}30 !important;
            }}
            
            .custom-radio input[type="radio"]:checked + span {{
                color: {COLORS['neon_purple']} !important;
                font-weight: 700 !important;
            }}
            
            /* CSS Animation for pulsing effect */
            @keyframes pulse {{
                0%, 100% {{ text-shadow: 0 0 20px currentColor, 0 0 40px currentColor; }}
                50% {{ text-shadow: 0 0 30px currentColor, 0 0 60px currentColor; }}
            }}
        </style>
    </head>
    <body>
        {{%app_entry%}}
        <footer>
            {{%config%}}
            {{%scripts%}}
            {{%renderer%}}
        </footer>
    </body>
</html>
'''

# ================================
# CALLBACKS
# ================================

# Loading screen callback - hides loading after components are ready
@app.callback(
    Output('loading-overlay', 'style'),
    [Input('sunburst-chart', 'figure'),
     Input('kpi-cards', 'children')]
)
def hide_loading(sunburst_fig, kpi_cards):
    """Hide loading screen once key components are loaded"""
    import time
    time.sleep(1)  # Brief delay to show the loading animation
    
    return {
        'position': 'fixed',
        'top': '0',
        'left': '0',
        'width': '100%',
        'height': '100%',
        'backgroundColor': 'rgba(10, 10, 26, 0.95)',
        'zIndex': '9999',
        'display': 'none',  # Hide the loading overlay
        'alignItems': 'center',
        'justifyContent': 'center',
        'backdropFilter': 'blur(10px)'
    }

# Populate course dropdown options
@app.callback(
    Output('course-dropdown', 'options'),
    Input('course-dropdown', 'id')
)
def populate_course_dropdown(_):
    """Populate course dropdown with available courses"""
    line_df = sem.dropna(subset=["GPA", "COURSE NAME", "PERIOD"]).copy()
    avg_gpa_by_semester = line_df.groupby(["PERIOD", "COURSE NAME"])["GPA"].agg(['mean', 'count']).reset_index()
    avg_gpa_by_semester.columns = ["PERIOD", "COURSE NAME", "Average_GPA", "Student_Count"]
    avg_gpa_by_semester = avg_gpa_by_semester[avg_gpa_by_semester["Student_Count"] >= 2].copy()
    
    unique_courses = sorted(avg_gpa_by_semester["COURSE NAME"].unique())
    options = [{'label': 'All Courses', 'value': 'all'}]
    options.extend([{'label': course, 'value': course} for course in unique_courses])
    return options

# Sunburst chart (Plotly Express)
@app.callback(
    Output('sunburst-chart', 'figure'),
    [Input('sunburst-chart', 'id'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_sunburst_chart(_, start_date, end_date):
    """Create sunburst chart showing citizenship vs course level vs course name with date filtering"""
    sun_df = profiles.dropna(subset=["COURSE LEVEL","COURSE NAME","CitizenshipSimple","COMMENCEMENT DATE"]).copy()
    
    # Filter by date range
    if start_date and end_date:
        sun_df = sun_df[
            (sun_df["COMMENCEMENT DATE"] >= pd.to_datetime(start_date)) & 
            (sun_df["COMMENCEMENT DATE"] <= pd.to_datetime(end_date))
        ]
    
    # Enhanced color scheme for better visibility
    enhanced_colors = {
        "SG Citizen": COLORS['neon_blue'],
        "SG PR": COLORS['neon_green'], 
        "Foreigner": COLORS['accent']
    }
    
    fig = px.sunburst(
        sun_df,
        path=["COURSE LEVEL", "COURSE NAME", "CitizenshipSimple"],
        color="CitizenshipSimple",
        color_discrete_map=enhanced_colors,
        title=""
    )
    
    # Enhanced styling for better readability
    fig.update_traces(
        hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percentParent}<extra></extra>",
        insidetextorientation="radial",
        textfont_size=12,
        textfont_color='white',
        textfont_family='Orbitron, monospace',
        textfont_weight='bold',
        marker_line_color='white',
        marker_line_width=2
    )
    
    fig.update_layout(
        margin=dict(l=10, r=10, t=10, b=10),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(color=COLORS['text'], family='Orbitron, monospace', size=12),
        height=450
    )
    return fig

# Date preset callback to update date range picker
@app.callback(
    [Output('date-range-picker', 'start_date'),
     Output('date-range-picker', 'end_date')],
    Input('date-preset-radio', 'value')
)
def update_date_range_from_preset(preset_value):
    """Update date range picker based on preset selection"""
    end_date = profiles['COMMENCEMENT DATE'].max()
    
    if preset_value == 'all':
        start_date = profiles['COMMENCEMENT DATE'].min()
    elif preset_value == 'last_2_years':
        start_date = end_date - pd.DateOffset(years=2)
    elif preset_value == 'last_5_years':
        start_date = end_date - pd.DateOffset(years=5)
    elif preset_value == 'since_2020':
        start_date = pd.to_datetime('2020-01-01')
    else:
        start_date = profiles['COMMENCEMENT DATE'].min()
    
    return start_date, end_date

# GPA trend line chart (Graph Objects with dropdown interaction)
@app.callback(
    Output('gpa-trend-chart', 'figure'),
    [Input('course-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_gpa_trend_chart(selected_course, start_date, end_date):
    """Create line chart showing GPA trends by semester with date filtering"""
    line_df = sem.dropna(subset=["GPA", "COURSE NAME", "PERIOD"]).copy()
    
    # Merge with profiles to get commencement dates for filtering
    line_df = line_df.merge(
        profiles[['STUDENT ID', 'COURSE CODE', 'COMMENCEMENT DATE']], 
        on=['STUDENT ID', 'COURSE CODE'], 
        how='left'
    )
    
    # Filter by date range
    if start_date and end_date:
        line_df = line_df[
            (line_df["COMMENCEMENT DATE"] >= pd.to_datetime(start_date)) & 
            (line_df["COMMENCEMENT DATE"] <= pd.to_datetime(end_date))
        ]
    avg_gpa_by_semester = line_df.groupby(["PERIOD", "COURSE NAME"])["GPA"].agg(['mean', 'count']).reset_index()
    avg_gpa_by_semester.columns = ["PERIOD", "COURSE NAME", "Average_GPA", "Student_Count"]
    avg_gpa_by_semester = avg_gpa_by_semester[avg_gpa_by_semester["Student_Count"] >= 2].copy()
    
    fig = go.Figure()
    
    if selected_course == 'all' or not selected_course:
        # Show all courses
        unique_courses = avg_gpa_by_semester["COURSE NAME"].unique()
        colors = px.colors.qualitative.Plotly
        
        for i, course in enumerate(unique_courses):
            course_data = avg_gpa_by_semester[avg_gpa_by_semester["COURSE NAME"] == course]
            fig.add_trace(go.Scatter(
                x=course_data["PERIOD"],
                y=course_data["Average_GPA"],
                mode='lines+markers',
                name=course,
                line=dict(width=3, color=colors[i % len(colors)]),
                marker=dict(size=8),
                hovertemplate="<b>" + course + "</b><br>Semester: %{x}<br>Average GPA: %{y:.3f}<br>Students: %{customdata}<extra></extra>",
                customdata=course_data["Student_Count"]
            ))
    else:
        # Show selected course only
        course_data = avg_gpa_by_semester[avg_gpa_by_semester["COURSE NAME"] == selected_course]
        fig.add_trace(go.Scatter(
            x=course_data["PERIOD"],
            y=course_data["Average_GPA"],
            mode='lines+markers',
            name=selected_course,
            line=dict(width=4, color=COLORS['neon_blue']),
            marker=dict(size=10),
            hovertemplate="<b>" + selected_course + "</b><br>Semester: %{x}<br>Average GPA: %{y:.3f}<br>Students: %{customdata}<extra></extra>",
            customdata=course_data["Student_Count"]
        ))
    
    semester_order = ["Semester 1", "Semester 2", "Semester 3", "Semester 4"]
    
    fig.update_layout(
        xaxis_title="Semester",
        yaxis_title="Average GPA",
        height=500,
        margin=dict(l=60, r=60, t=20, b=60),
        legend_title_text="Course Name",
        hovermode="x unified",
        xaxis=dict(
            categoryorder="array",
            categoryarray=semester_order
        ),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(color=COLORS['text']),
        legend=dict(font=dict(color=COLORS['text']))
    )
    return fig

# Violin chart (Graph Objects with radio button interaction)
@app.callback(
    Output('violin-chart', 'figure'),
    [Input('age-group-radio', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_violin_chart(selected_age_group, start_date, end_date):
    """Create violin chart showing age distribution by course name with date filtering"""
    v_df = profiles.dropna(subset=["DOB", "COMMENCEMENT DATE", "COURSE NAME"]).copy()
    
    # Filter by date range
    if start_date and end_date:
        v_df = v_df[
            (v_df["COMMENCEMENT DATE"] >= pd.to_datetime(start_date)) & 
            (v_df["COMMENCEMENT DATE"] <= pd.to_datetime(end_date))
        ]
    
    v_df["Age"] = (v_df["COMMENCEMENT DATE"] - v_df["DOB"]).dt.days / 365.25
    v_df = v_df[(v_df["Age"] >= 15) & (v_df["Age"] <= 80)].copy()
    
    # Filter by age group
    if selected_age_group == 'young':
        age_group_data = v_df[v_df["Age"] <= 25]
        title_suffix = "Young Adults (15-25)"
    elif selected_age_group == 'adult':
        age_group_data = v_df[(v_df["Age"] > 25) & (v_df["Age"] <= 40)]
        title_suffix = "Adults (26-40)"
    elif selected_age_group == 'mature':
        age_group_data = v_df[v_df["Age"] > 40]
        title_suffix = "Mature Adults (41-80)"
    else:
        age_group_data = v_df
        title_suffix = "All Ages (15-80)"
    
    fig = go.Figure()
    
    fig.add_trace(go.Violin(
        x=age_group_data["COURSE NAME"],
        y=age_group_data["Age"],
        name=title_suffix,
        box_visible=True,
        meanline_visible=True,
        points=False,
        jitter=0.3,
        scalemode="count",
        line_color=COLORS['neon_green'],
        fillcolor=COLORS['neon_green'],
        opacity=0.6,
        hovertemplate="<b>%{x}</b><br>Age at enrollment: %{y:.1f} years<extra></extra>"
    ))
    
    fig.update_layout(
        xaxis_title="Course Name",
        yaxis_title="Age at Enrollment (years)",
        height=500,
        margin=dict(l=60, r=60, t=20, b=100),
        xaxis=dict(tickangle=-45),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(color=COLORS['text']),
        showlegend=False
    )
    return fig

# Scatter chart (Graph Objects with dropdown interaction)
@app.callback(
    Output('scatter-chart', 'figure'),
    [Input('course-level-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_scatter_chart(selected_level, start_date, end_date):
    """Create scatter chart showing course popularity by gender with date filtering"""
    p_df = profiles.dropna(subset=["Gender","COURSE NAME","COMMENCEMENT DATE"]).copy()
    
    # Filter by date range
    if start_date and end_date:
        p_df = p_df[
            (p_df["COMMENCEMENT DATE"] >= pd.to_datetime(start_date)) & 
            (p_df["COMMENCEMENT DATE"] <= pd.to_datetime(end_date))
        ]
    
    # Calculate total students by gender
    total_females = len(p_df[p_df["Gender"] == "Female"])
    total_males = len(p_df[p_df["Gender"] == "Male"])
    
    # Calculate course enrollment by gender
    course_gender_counts = p_df.groupby(["COURSE NAME", "Gender"]).size().unstack(fill_value=0)
    
    # Calculate percentages
    course_gender_pct = course_gender_counts.copy()
    if "Female" in course_gender_pct.columns:
        course_gender_pct["Female_Pct"] = (course_gender_pct["Female"] / total_females) * 100
    else:
        course_gender_pct["Female_Pct"] = 0
        
    if "Male" in course_gender_pct.columns:
        course_gender_pct["Male_Pct"] = (course_gender_pct["Male"] / total_males) * 100
    else:
        course_gender_pct["Male_Pct"] = 0
    
    course_popularity = course_gender_pct.reset_index()
    
    # Add course level information
    course_levels = []
    for course in course_popularity["COURSE NAME"]:
        if course.startswith("Certificate"):
            course_levels.append("Certificate")
        elif course.startswith("Diploma") and not course.startswith("Specialist"):
            course_levels.append("Diploma") 
        elif course.startswith("Specialist"):
            course_levels.append("Specialist Diploma")
        else:
            course_levels.append("Other")
    
    course_popularity["COURSE_LEVEL"] = course_levels
    
    # Filter by course level
    if selected_level != 'all':
        filtered_data = course_popularity[course_popularity["COURSE_LEVEL"] == selected_level]
    else:
        filtered_data = course_popularity
    
    fig = go.Figure()
    
    colors = px.colors.qualitative.Plotly
    for i, (_, row) in enumerate(filtered_data.iterrows()):
        fig.add_trace(go.Scatter(
            x=[row["Female_Pct"]],
            y=[row["Male_Pct"]],
            mode='markers',
            name=row["COURSE NAME"],
            marker=dict(
                size=12,
                color=colors[i % len(colors)],
                opacity=0.8
            ),
            hovertemplate=f"<b>{row['COURSE NAME']}</b><br>Female: {row['Female_Pct']:.1f}% ({row['Female']} students)<br>Male: {row['Male_Pct']:.1f}% ({row['Male']} students)<extra></extra>"
        ))
    
    # Add reference line
    if not filtered_data.empty:
        max_pct = max(filtered_data["Female_Pct"].max(), filtered_data["Male_Pct"].max())
        fig.add_shape(
            type="line",
            x0=0, y0=0,
            x1=max_pct, y1=max_pct,
            line=dict(color="gray", width=1, dash="dash"),
            opacity=0.5
        )
    
    fig.update_layout(
        xaxis=dict(
            title="Female Popularity (%)",
            tickformat=".1f"
        ),
        yaxis=dict(
            title="Male Popularity (%)",
            tickformat=".1f"
        ),
        margin=dict(l=60, r=60, t=20, b=60),
        legend_title_text="Course Name",
        height=500,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(color=COLORS['text']),
        legend=dict(font=dict(color=COLORS['text']))
    )
    return fig

# KPI cards - reactive to all filters
@app.callback(
    Output('kpi-cards', 'children'),
    [Input('course-dropdown', 'value'),
     Input('age-group-radio', 'value'),
     Input('course-level-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_kpi_cards(selected_course, selected_age_group, selected_level, start_date, end_date):
    """Update KPI cards based on all filter selections including dates"""
    
    # Filter profiles data based on date range first
    kpi_profiles = profiles.dropna(subset=["DOB", "COMMENCEMENT DATE"]).copy()
    
    # Apply date range filter
    if start_date and end_date:
        kpi_profiles = kpi_profiles[
            (kpi_profiles["COMMENCEMENT DATE"] >= pd.to_datetime(start_date)) & 
            (kpi_profiles["COMMENCEMENT DATE"] <= pd.to_datetime(end_date))
        ]
    kpi_profiles["Age"] = (kpi_profiles["COMMENCEMENT DATE"] - kpi_profiles["DOB"]).dt.days / 365.25
    kpi_profiles = kpi_profiles[(kpi_profiles["Age"] >= 15) & (kpi_profiles["Age"] <= 80)].copy()
    
    # Apply age group filter
    if selected_age_group == 'young':
        kpi_profiles = kpi_profiles[kpi_profiles["Age"] <= 25]
    elif selected_age_group == 'adult':
        kpi_profiles = kpi_profiles[(kpi_profiles["Age"] > 25) & (kpi_profiles["Age"] <= 40)]
    elif selected_age_group == 'mature':
        kpi_profiles = kpi_profiles[kpi_profiles["Age"] > 40]
    
    # Apply course level filter
    if selected_level != 'all':
        kpi_profiles = kpi_profiles[kpi_profiles["COURSE LEVEL"] == selected_level]
    
    # Filter semester data based on course selection
    kpi_sem = sem.copy()
    if selected_course != 'all' and selected_course:
        kpi_sem = kpi_sem[kpi_sem["COURSE NAME"] == selected_course]
    
    # Calculate KPIs
    total_students = len(kpi_profiles)
    avg_gpa = kpi_sem['GPA'].mean() if not kpi_sem.empty else 0
    employment_rate = (kpi_profiles['EmployedFlag'].sum() / len(kpi_profiles) * 100) if len(kpi_profiles) > 0 else 0
    avg_age = kpi_profiles['Age'].mean() if not kpi_profiles.empty else 0
    
    # Create KPI cards with horizontal layout
    kpi_cards = [
        # Total Students
        html.Div([
            html.Div([
                html.H3(f"{total_students:,}", style={
                    'color': COLORS['neon_blue'], 'margin': '0', 'fontSize': '2.2rem',
                    'fontFamily': 'Orbitron, monospace', 'fontWeight': '900',
                    'textShadow': f'0 0 10px {COLORS["neon_blue"]}'
                }),
                html.P("Total Students", style={
                    'color': COLORS['text_dim'], 'margin': '8px 0 0 0', 'fontSize': '0.9rem',
                    'fontFamily': 'Rajdhani, monospace', 'textTransform': 'uppercase',
                    'fontWeight': '600'
                })
            ], style={'textAlign': 'center'})
        ], style={
            'background': f'linear-gradient(135deg, {COLORS["neon_blue"]}25 0%, {COLORS["card_bg"]} 100%)',
            'border': f'2px solid {COLORS["neon_blue"]}70',
            'borderRadius': '15px', 'padding': '25px 20px',
            'boxShadow': f'0 0 20px {COLORS["neon_blue"]}40, 0 8px 32px rgba(26, 26, 46, 0.6)',
            'transition': 'all 0.3s ease',
            'minHeight': '120px',
            'display': 'flex',
            'alignItems': 'center',
            'justifyContent': 'center'
        }),
        
        # Average GPA
        html.Div([
            html.Div([
                html.H3(f"{avg_gpa:.2f}", style={
                    'color': COLORS['warning'], 'margin': '0', 'fontSize': '2.2rem',
                    'fontFamily': 'Orbitron, monospace', 'fontWeight': '900',
                    'textShadow': f'0 0 10px {COLORS["warning"]}'
                }),
                html.P("Average GPA", style={
                    'color': COLORS['text_dim'], 'margin': '8px 0 0 0', 'fontSize': '0.9rem',
                    'fontFamily': 'Rajdhani, monospace', 'textTransform': 'uppercase',
                    'fontWeight': '600'
                })
            ], style={'textAlign': 'center'})
        ], style={
            'background': f'linear-gradient(135deg, {COLORS["warning"]}25 0%, {COLORS["card_bg"]} 100%)',
            'border': f'2px solid {COLORS["warning"]}70',
            'borderRadius': '15px', 'padding': '25px 20px',
            'boxShadow': f'0 0 20px {COLORS["warning"]}40, 0 8px 32px rgba(26, 26, 46, 0.6)',
            'transition': 'all 0.3s ease',
            'minHeight': '120px',
            'display': 'flex',
            'alignItems': 'center',
            'justifyContent': 'center'
        }),
        
        # Employment Rate
        html.Div([
            html.Div([
                html.H3(f"{employment_rate:.1f}%", style={
                    'color': COLORS['success'], 'margin': '0', 'fontSize': '2.2rem',
                    'fontFamily': 'Orbitron, monospace', 'fontWeight': '900',
                    'textShadow': f'0 0 10px {COLORS["success"]}'
                }),
                html.P("Employment Rate", style={
                    'color': COLORS['text_dim'], 'margin': '8px 0 0 0', 'fontSize': '0.9rem',
                    'fontFamily': 'Rajdhani, monospace', 'textTransform': 'uppercase',
                    'fontWeight': '600'
                })
            ], style={'textAlign': 'center'})
        ], style={
            'background': f'linear-gradient(135deg, {COLORS["success"]}25 0%, {COLORS["card_bg"]} 100%)',
            'border': f'2px solid {COLORS["success"]}70',
            'borderRadius': '15px', 'padding': '25px 20px',
            'boxShadow': f'0 0 20px {COLORS["success"]}40, 0 8px 32px rgba(26, 26, 46, 0.6)',
            'transition': 'all 0.3s ease',
            'minHeight': '120px',
            'display': 'flex',
            'alignItems': 'center',
            'justifyContent': 'center'
        }),
        
        # Average Age
        html.Div([
            html.Div([
                html.H3(f"{avg_age:.1f}", style={
                    'color': COLORS['accent'], 'margin': '0', 'fontSize': '2.2rem',
                    'fontFamily': 'Orbitron, monospace', 'fontWeight': '900',
                    'textShadow': f'0 0 10px {COLORS["accent"]}'
                }),
                html.P("Average Age", style={
                    'color': COLORS['text_dim'], 'margin': '8px 0 0 0', 'fontSize': '0.9rem',
                    'fontFamily': 'Rajdhani, monospace', 'textTransform': 'uppercase',
                    'fontWeight': '600'
                })
            ], style={'textAlign': 'center'})
        ], style={
            'background': f'linear-gradient(135deg, {COLORS["accent"]}25 0%, {COLORS["card_bg"]} 100%)',
            'border': f'2px solid {COLORS["accent"]}70',
            'borderRadius': '15px', 'padding': '25px 20px',
            'boxShadow': f'0 0 20px {COLORS["accent"]}40, 0 8px 32px rgba(26, 26, 46, 0.6)',
            'transition': 'all 0.3s ease',
            'minHeight': '120px',
            'display': 'flex',
            'alignItems': 'center',
            'justifyContent': 'center'
        })
    ]
    
    return kpi_cards

# Student data display with beautiful cards
@app.callback(
    Output('student-data-table', 'children'),
    Input('search-box', 'value')
)
def update_student_display(search_value):
    """Update student data display with beautiful formatted cards"""
    if not search_value or len(search_value.strip()) < 3:
        return html.Div([
            html.Div([
                html.I(className="fas fa-search", style={
                    'fontSize': '3rem', 'color': COLORS['neon_blue'], 'marginBottom': '15px'
                }),
                html.H4("🔍 STUDENT SEARCH", style={
                    'color': COLORS['neon_blue'], 'fontFamily': 'Orbitron, monospace',
                    'textAlign': 'center', 'marginBottom': '10px'
                }),
                html.P("Enter at least 3 characters to search for a student ID", 
                       style={
                           'textAlign': 'center', 
                           'color': COLORS['text_dim'], 
                           'fontSize': '0.9rem',
                           'fontFamily': 'Rajdhani, monospace',
                           'fontStyle': 'italic'
                       })
            ], style={
                'textAlign': 'center',
                'padding': '40px 20px',
                'background': f'linear-gradient(135deg, {COLORS["card_bg"]}40 0%, {COLORS["sidebar_bg"]}40 100%)',
                'borderRadius': '15px',
                'border': f'1px dashed {COLORS["neon_blue"]}60'
            })
        ])
    
    # Search for students
    student_data = profiles[
        profiles['STUDENT ID'].str.contains(search_value, case=False, na=False)
    ]
    
    if student_data.empty:
        return html.Div([
            html.Div([
                html.I(className="fas fa-exclamation-triangle", style={
                    'fontSize': '2.5rem', 'color': COLORS['danger'], 'marginBottom': '15px'
                }),
                html.H4("❌ NO RESULTS", style={
                    'color': COLORS['danger'], 'fontFamily': 'Orbitron, monospace',
                    'textAlign': 'center', 'marginBottom': '10px'
                }),
                html.P(f"No student found with ID containing '{search_value}'", 
                       style={
                           'textAlign': 'center', 
                           'color': COLORS['text'], 
                           'fontSize': '1rem',
                           'fontFamily': 'Rajdhani, monospace'
                       })
            ], style={
                'textAlign': 'center',
                'padding': '30px 20px',
                'background': f'linear-gradient(135deg, {COLORS["danger"]}20 0%, {COLORS["card_bg"]} 100%)',
                'borderRadius': '15px',
                'border': f'2px solid {COLORS["danger"]}60'
            })
        ])
    
    # Create beautiful student information cards
    student_cards = []
    for _, student in student_data.head(3).iterrows():  # Limit to 3 detailed results
        
        # Get semester data for this student
        student_sem_data = sem[sem['STUDENT ID'] == student['STUDENT ID']]
        avg_gpa = student_sem_data['GPA'].mean() if not student_sem_data.empty else 0
        total_semesters = len(student_sem_data) if not student_sem_data.empty else 0
        
        # Create individual student card
        card = html.Div([
            # Header with student ID
            html.Div([
                html.H4(f"👤 {student['STUDENT ID']}", style={
                    'color': COLORS['neon_blue'], 'margin': '0', 'fontFamily': 'Orbitron, monospace',
                    'fontSize': '1.3rem', 'fontWeight': '900', 'textAlign': 'center',
                    'textShadow': f'0 0 10px {COLORS["neon_blue"]}'
                })
            ], style={
                'background': f'linear-gradient(90deg, {COLORS["neon_blue"]}20 0%, {COLORS["neon_purple"]}20 100%)',
                'padding': '15px', 'borderRadius': '10px 10px 0 0',
                'borderBottom': f'2px solid {COLORS["neon_blue"]}'
            }),
            
            # Main content in two columns
            html.Div([
                # Left column - Personal Info
                html.Div([
                    html.H5("📋 PERSONAL INFO", style={
                        'color': COLORS['neon_green'], 'marginBottom': '15px',
                        'fontFamily': 'Orbitron, monospace', 'fontSize': '0.9rem'
                    }),
                    
                    # Personal details with icons
                    html.Div([
                        html.Span("👤 ", style={'color': COLORS['accent']}),
                        html.Span("Gender: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span(str(student.get('Gender', 'N/A')), style={'color': COLORS['text'], 'fontWeight': '700'})
                    ], style={'marginBottom': '8px', 'fontFamily': 'Rajdhani, monospace'}),
                    
                    html.Div([
                        html.Span("🌍 ", style={'color': COLORS['accent']}),
                        html.Span("Citizenship: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span(str(student.get('CitizenshipSimple', 'N/A')), style={'color': COLORS['neon_purple'], 'fontWeight': '700'})
                    ], style={'marginBottom': '8px', 'fontFamily': 'Rajdhani, monospace'}),
                    
                    html.Div([
                        html.Span("🎂 ", style={'color': COLORS['accent']}),
                        html.Span("Age: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span(f"{student.get('AGE_AT_QUALIFICATION', 0):.0f} years", style={'color': COLORS['text'], 'fontWeight': '700'})
                    ], style={'marginBottom': '8px', 'fontFamily': 'Rajdhani, monospace'}),
                    
                    html.Div([
                        html.Span("💼 ", style={'color': COLORS['accent']}),
                        html.Span("Status: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span("Employed" if student.get('EmployedFlag', False) else "Unemployed", 
                                 style={'color': COLORS['success'] if student.get('EmployedFlag', False) else COLORS['danger'], 'fontWeight': '700'})
                    ], style={'marginBottom': '15px', 'fontFamily': 'Rajdhani, monospace'})
                    
                ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                
                # Right column - Academic Info
                html.Div([
                    html.H5("🎓 ACADEMIC INFO", style={
                        'color': COLORS['neon_purple'], 'marginBottom': '15px',
                        'fontFamily': 'Orbitron, monospace', 'fontSize': '0.9rem'
                    }),
                    
                    # Academic details with icons
                    html.Div([
                        html.Span("🏫 ", style={'color': COLORS['accent']}),
                        html.Span("Department: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span(str(student.get('DEPARTMENT', 'N/A'))[:20] + ("..." if len(str(student.get('DEPARTMENT', ''))) > 20 else ""), 
                                 style={'color': COLORS['text'], 'fontWeight': '700'})
                    ], style={'marginBottom': '8px', 'fontFamily': 'Rajdhani, monospace'}),
                    
                    html.Div([
                        html.Span("📚 ", style={'color': COLORS['accent']}),
                        html.Span("Course: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span(str(student.get('COURSE NAME', 'N/A'))[:25] + ("..." if len(str(student.get('COURSE NAME', ''))) > 25 else ""), 
                                 style={'color': COLORS['neon_green'], 'fontWeight': '700'})
                    ], style={'marginBottom': '8px', 'fontFamily': 'Rajdhani, monospace'}),
                    
                    html.Div([
                        html.Span("⭐ ", style={'color': COLORS['accent']}),
                        html.Span("GPA: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span(f"{avg_gpa:.2f}", style={'color': COLORS['warning'], 'fontWeight': '900', 'fontSize': '1.1rem'})
                    ], style={'marginBottom': '8px', 'fontFamily': 'Rajdhani, monospace'}),
                    
                    html.Div([
                        html.Span("📊 ", style={'color': COLORS['accent']}),
                        html.Span("Semesters: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span(f"{total_semesters}", style={'color': COLORS['info'], 'fontWeight': '700'})
                    ], style={'marginBottom': '8px', 'fontFamily': 'Rajdhani, monospace'}),
                    
                    html.Div([
                        html.Span("💰 ", style={'color': COLORS['accent']}),
                        html.Span("Funding: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                        html.Span(str(student.get('COURSE FUNDING', 'N/A')), style={'color': COLORS['text'], 'fontWeight': '700'})
                    ], style={'marginBottom': '15px', 'fontFamily': 'Rajdhani, monospace'})
                    
                ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'marginLeft': '4%'})
                
            ], style={'padding': '20px'}),
            
            # Footer with qualification info
            html.Div([
                html.Span("🏆 Highest Qualification: ", style={'color': COLORS['text_dim'], 'fontWeight': '500'}),
                html.Span(str(student.get('HIGHEST QUALIFICATION', 'N/A')), 
                         style={'color': COLORS['success'], 'fontWeight': '700', 'fontSize': '1rem'})
            ], style={
                'background': f'linear-gradient(90deg, {COLORS["success"]}20 0%, {COLORS["card_bg"]} 100%)',
                'padding': '12px 20px', 'borderRadius': '0 0 10px 10px',
                'borderTop': f'1px solid {COLORS["success"]}40',
                'fontFamily': 'Rajdhani, monospace'
            })
            
        ], style={
            'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)',
            'border': f'2px solid {COLORS["neon_blue"]}60',
            'borderRadius': '15px',
            'marginBottom': '20px',
            'boxShadow': f'0 0 30px {COLORS["neon_blue"]}30, 0 8px 32px rgba(26, 26, 46, 0.6)',
            'transition': 'all 0.3s ease',
            'overflow': 'hidden'
        })
        
        student_cards.append(card)
    
    # Results summary
    results_summary = html.Div([
        html.H4(f"✅ FOUND {len(student_data)} STUDENT(S)", style={
            'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '5px',
            'fontFamily': 'Orbitron, monospace', 'fontSize': '1rem'
        }),
        html.P(f"Search term: '{search_value}' | Showing detailed info for {min(len(student_data), 3)} student(s)", 
               style={
                   'color': COLORS['text_dim'], 'textAlign': 'center', 'marginBottom': '20px',
                   'fontFamily': 'Rajdhani, monospace', 'fontSize': '0.9rem'
               })
    ])
    
    return html.Div([results_summary] + student_cards)

# ================================
# RUN THE APP
# ================================

if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=8055)

🚀 Loading Singapore student pathway data...
✅ Successfully loaded:
   📊 285 student profiles
   📈 531 semester records
   📚 7 course codes
